# **RA1: data 230705**

Dong Gyun Ko <br/>
last updated: july 5, 2023 <br/>

In [230]:
# import libraries

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib.pylab as plt
import datetime as dt
import os
import warnings
import requests # python 3.6
from tqdm import tqdm

import statsmodels.api as sm
from statsmodels.tsa.seasonal import STL

import pandas_datareader as pdr
import pandas_datareader.data as web
from pandas_datareader import wb
from pandas_datareader.data import DataReader

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [231]:
# set the working directory

os.chdir('/content/gdrive/MyDrive/Colab Notebooks/RA1_updated_data')

In [232]:
# set the pandas option

pd.reset_option('^display.', silent=True)
# pd.set_option('display.max_rows', None)

In [233]:
# seasonal adjustment function

# classical decomposition
def cd(df):

    seasonal = sm.tsa.seasonal_decompose(df, model='additive', period=4).seasonal

    return df - seasonal

# STL decomposition
def stld(df):

    stl = STL(df, period=4, seasonal=5)
    res = stl.fit()
    seasonal = res.seasonal

    return df - seasonal

In [234]:
# period function by var (46 countries)

# period func.
def period_46_countries(var):

    for i in range(0, data['isocode'].unique().shape[0]):

        globals()['cond_isocode_{}'.format(i+1)] = (data['isocode'] == data['isocode'].unique()[i])
        globals()['df_isocode_{}'.format(i+1)] = data.loc[globals()['cond_isocode_{}'.format(i+1)]]
        globals()['df_isocode_count_T_{}'.format(i+1)] = globals()['df_isocode_{}'.format(i+1)].groupby(globals()['df_isocode_{}'.format(i+1)]['time']).count()

        globals()['cond_{}'.format(var) + '_{}'.format(i+1)] = (globals()['df_isocode_count_T_{}'.format(i+1)][var] == 0)
        globals()['df_isocode_count_T_{}'.format(i+1)] = globals()['df_isocode_count_T_{}'.format(i+1)].loc[~globals()['cond_{}'.format(var) + '_{}'.format(i+1)]]
        globals()['df_isocode_count_T_{}'.format(i+1)] = globals()['df_isocode_count_T_{}'.format(i+1)].reset_index()

        # missing
        if globals()['df_isocode_count_T_{}'.format(i+1)]['time'].unique().shape[0] == 0:

           globals()['tab_period_{}'.format(var) + '_{}'.format(i+1)] = pd.DataFrame(np.array([data['country'].unique()[i],
                                                                                              data['isocode'].unique()[i],
                                                                                              np.nan,
                                                                                              np.nan]).reshape(1, -1)
                                                                                              ).rename(columns={0: 'country',
                                                                                                                1: 'isocode',
                                                                                                                2: 'period',
                                                                                                                3: 'obs'}).reset_index(drop=True)

        # observed
        else:

           globals()['tab_period_{}'.format(var) + '_{}'.format(i+1)] = pd.DataFrame(np.array([data['country'].unique()[i],
                                                                                              data['isocode'].unique()[i],
                                                                                              str(globals()['df_isocode_count_T_{}'.format(i+1)]['time'].unique().min()) + '-' + str(globals()['df_isocode_count_T_{}'.format(i+1)]['time'].unique().max()),
                                                                                              globals()['df_isocode_count_T_{}'.format(i+1)][var].count()]).reshape(1, -1)
                                                                                              ).rename(columns={0: 'country',
                                                                                                                1: 'isocode',
                                                                                                                2: 'period',
                                                                                                                3: 'obs'}).reset_index(drop=True)

    globals()['tab_period_{}'.format(var)] = pd.concat([globals()['tab_period_{}'.format(var) + '_{}'.format(1)], globals()['tab_period_{}'.format(var) + '_{}'.format(2)]])

    for j in range(3, data['isocode'].unique().shape[0]+1):

        globals()['tab_period_{}'.format(var)] = pd.concat([globals()['tab_period_{}'.format(var)], globals()['tab_period_{}'.format(var) + '_{}'.format(j)]])

    globals()['tab_period_{}'.format(var)] = pd.DataFrame(globals()['tab_period_{}'.format(var)]).reset_index(drop=True)

    return globals()['tab_period_{}'.format(var)]

# period var func.
def period_var_46_countries(var):

    isocode_list = ['ARG', 'AUS', 'AUT', 'BEL', 'BGR', 'BRA', 'CAN', 'CHE', 'CHL', 'COL',
                    'CZE', 'DEU', 'DNK', 'ESP', 'EST', 'FIN', 'FRA', 'GBR', 'GRC', 'HKG',
                    'HRV', 'HUN', 'IDN', 'IND', 'IRL', 'ISR', 'ITA', 'JPN', 'KOR', 'LTU',
                    'LVA', 'MEX', 'NLD', 'NOR', 'NZL', 'POL', 'PRT', 'ROU', 'RUS', 'SVK',
                    'SVN', 'SWE', 'THA', 'TUR', 'USA', 'ZAF']

    tab_period_var = period_46_countries(var)

    for i in range(0, len(isocode_list)):

        globals()['cond_N{}'.format(i+1)] = (tab_period_var['isocode'] == isocode_list[i])

    tab_period_var = tab_period_var.loc[cond_N1 | cond_N2 | cond_N3 | cond_N4 | cond_N5 | cond_N6 | cond_N7 | cond_N8 | cond_N9 | cond_N10 |
                                        cond_N11 | cond_N12 | cond_N13 | cond_N14 | cond_N15 | cond_N16 | cond_N17 | cond_N18 | cond_N19 | cond_N20 |
                                        cond_N21 | cond_N22 | cond_N23 | cond_N24 | cond_N25 | cond_N26 | cond_N27 | cond_N28 | cond_N29 | cond_N30 |
                                        cond_N31 | cond_N32 | cond_N33 | cond_N34 | cond_N35 | cond_N36 | cond_N37 | cond_N38 | cond_N39 | cond_N40 |
                                        cond_N41 | cond_N42 | cond_N43 | cond_N44 | cond_N45 | cond_N46]

    tab_period_var = tab_period_var.sort_values(by=['isocode']).reset_index(drop=True)

    return tab_period_var

# **1. df_baseline**

In [235]:
# generate df_baseline

df_baseline = pd.read_csv('df_q_temp.csv').set_index(['country', 'alpha-3']).transpose()
df_baseline = pd.DataFrame(df_baseline.stack(level=['country', 'alpha-3'])).reset_index()
df_baseline = df_baseline.rename(columns={'level_0':'time', 'alpha-3':'isocode', 0:'value'})
df_baseline = df_baseline[['country', 'isocode', 'time', 'value']].sort_values(by=['isocode', 'time']).reset_index(drop=True)
df_baseline['time'] = pd.to_datetime(df_baseline['time'])
df_baseline['time'] = df_baseline['time'].dt.to_period('Q')

<ipython-input-235-a892922d92d4>:7: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df_baseline['time'] = pd.to_datetime(df_baseline['time'])


In [236]:
# select the T and N

# select the T
cond_T_start = (df_baseline['time'].dt.year >= 1950)
cond_T_end = (df_baseline['time'].dt.year < 2014)

df_baseline = df_baseline.loc[(cond_T_start & cond_T_end)]

# select the N
isocode_array = np.array(['ARG', 'AUS', 'AUT', 'BEL', 'BGR', 'BRA', 'CAN', 'CHE', 'CHL', 'COL',
                          'CZE', 'DEU', 'DNK', 'ESP', 'EST', 'FIN', 'FRA', 'GBR', 'GRC', 'HKG',
                          'HRV', 'HUN', 'IDN', 'IND', 'IRL', 'ISR', 'ITA', 'JPN', 'KOR', 'LTU',
                          'LVA', 'MEX', 'NLD', 'NOR', 'NZL', 'POL', 'PRT', 'ROU', 'RUS', 'SVK',
                          'SVN', 'SWE', 'THA', 'TUR', 'USA', 'ZAF'])

for i in range(0, isocode_array.shape[0]):

    globals()['cond_N{}'.format(i+1)] = (df_baseline['isocode'] == isocode_array[i])

df_baseline = df_baseline.loc[cond_N1 | cond_N2 | cond_N3 | cond_N4 | cond_N5 | cond_N6 | cond_N7 | cond_N8 | cond_N9 | cond_N10 |
                              cond_N11 | cond_N12 | cond_N13 | cond_N14 | cond_N15 | cond_N16 | cond_N17 | cond_N18 | cond_N19 | cond_N20 |
                              cond_N21 | cond_N22 | cond_N23 | cond_N24 | cond_N25 | cond_N26 | cond_N27 | cond_N28 | cond_N29 | cond_N30 |
                              cond_N31 | cond_N32 | cond_N33 | cond_N34 | cond_N35 | cond_N36 | cond_N37 | cond_N38 | cond_N39 | cond_N40 |
                              cond_N41 | cond_N42 | cond_N43 | cond_N44 | cond_N45 | cond_N46]

In [237]:
# df_baseline

df_baseline = df_baseline.sort_values(by=['isocode', 'time']).reset_index(drop=True)
df_baseline

,country,isocode,time,value
0,Argentina,ARG,1950Q1,0.0
1,Argentina,ARG,1950Q2,0.0
2,Argentina,ARG,1950Q3,0.0
3,Argentina,ARG,1950Q4,0.0
4,Argentina,ARG,1951Q1,0.0
...,...,...,...,...
11771,South Africa,ZAF,2012Q4,0.0
11772,South Africa,ZAF,2013Q1,0.0
11773,South Africa,ZAF,2013Q2,0.0
11774,South Africa,ZAF,2013Q3,0.0


# **2. data 221115**

In [238]:
# import the data

data_221115 = pd.read_excel('data_221115.xlsx').sort_values(by=['country', 'quarter']).reset_index(drop=True)
df_time = pd.read_excel('df_time.xlsx').reset_index(drop=True)

In [239]:
# merge the quarterly pooled data with the quarterly temp data

data_221115 = pd.merge(data_221115, df_time, how='outer', on=['quarter'])

In [240]:
# datetime

data_221115['time'] = pd.to_datetime(data_221115['time'])
data_221115['time'] = data_221115['time'].dt.to_period('Q')

In [241]:
# sorting

data_221115 = data_221115.sort_values(by=['isocode', 'quarter']).reset_index(drop=True) # sort by isocode and quarter

In [242]:
# ordering

data_221115 = data_221115[['country', 'isocode', 'time', 'quarter',
                           'ccode', 'region', 'income', 'oecd', 'g7',
                           'ggdebt', 'ggcon', 'gdp', 'reer', 'ca', 'pcon', 'cds',
                           'sample2', 'ex', 'inc_ex', 'debtlevel', 'inc_debt', 'tradeopen', 'inc_trade', 'fin', 'inc_fin', 'inc_fine', 'inc_debt_fine', 'finopen_debt', 'develop', 'inv', 'vehicle', 'vehicle2', 'adveme']]

In [243]:
# merge data_221115 with df_baseline

data_221115  = pd.merge(data_221115 , df_baseline, how='right', on=['country', 'isocode', 'time']).drop(columns=['value'])

In [244]:
# data_221115

data_221115 = data_221115.sort_values(by=['isocode', 'time']).reset_index(drop=True)
data_221115

,country,isocode,time,quarter,ccode,region,income,oecd,g7,ggdebt,...,fin,inc_fin,inc_fine,inc_debt_fine,finopen_debt,develop,inv,vehicle,vehicle2,adveme
0,Argentina,ARG,1950Q1,1,32,7,2,0,0,NaN,...,1,3,2,4,2,2,NaN,0,0,2
1,Argentina,ARG,1950Q2,2,32,7,2,0,0,NaN,...,1,3,2,4,2,2,NaN,0,0,2
2,Argentina,ARG,1950Q3,3,32,7,2,0,0,NaN,...,1,3,2,4,2,2,NaN,0,0,2
3,Argentina,ARG,1950Q4,4,32,7,2,0,0,NaN,...,1,3,2,4,2,2,NaN,0,0,2
4,Argentina,ARG,1951Q1,5,32,7,2,0,0,NaN,...,1,3,2,4,2,2,NaN,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11771,South Africa,ZAF,2012Q4,252,39,5,2,0,0,39.019756,...,1,3,3,5,1,2,1.912744e+11,0,0,2
11772,South Africa,ZAF,2013Q1,253,39,5,2,0,0,39.796563,...,1,3,3,5,1,2,1.943342e+11,0,0,2
11773,South Africa,ZAF,2013Q2,254,39,5,2,0,0,40.444657,...,1,3,3,5,1,2,1.983998e+11,0,0,2
11774,South Africa,ZAF,2013Q3,255,39,5,2,0,0,41.127999,...,1,3,3,5,1,2,2.026111e+11,0,0,2


In [245]:
# set the data

data = data_221115

In [246]:
# ggdebt

period_var_46_countries('ggdebt')

,country,isocode,period,obs
0,Argentina,ARG,1992Q4-2013Q4,85
1,Australia,AUS,1988Q2-2013Q4,103
2,Austria,AUT,2000Q1-2013Q4,56
3,Belgium,BEL,1995Q1-2013Q4,76
4,Bulgaria,BGR,2000Q1-2013Q4,56
5,Brazil,BRA,1998Q1-2013Q4,64
6,Canada,CAN,1990Q1-2013Q4,96
7,Switzerland,CHE,1995Q1-2013Q4,76
8,Chile,CHL,1998Q1-2013Q4,64
9,Colombia,COL,1996Q4-2013Q4,69


In [247]:
# gdp

period_var_46_countries('gdp')

,country,isocode,period,obs
0,Argentina,ARG,2004Q1-2013Q4,40
1,Australia,AUS,1980Q1-2013Q4,136
2,Austria,AUT,1995Q1-2013Q4,76
3,Belgium,BEL,1995Q1-2013Q4,76
4,Bulgaria,BGR,1999Q1-2013Q4,60
5,Brazil,BRA,1996Q1-2013Q4,72
6,Canada,CAN,1980Q1-2013Q4,136
7,Switzerland,CHE,1980Q1-2013Q4,136
8,Chile,CHL,1996Q1-2013Q4,72
9,Colombia,COL,2005Q1-2013Q4,36


In [248]:
# ggcon

period_var_46_countries('ggcon')

,country,isocode,period,obs
0,Argentina,ARG,2004Q1-2013Q4,40
1,Australia,AUS,1980Q1-2013Q4,136
2,Austria,AUT,1995Q1-2013Q4,76
3,Belgium,BEL,1995Q1-2013Q4,76
4,Bulgaria,BGR,1999Q1-2013Q4,60
5,Brazil,BRA,1996Q1-2013Q4,72
6,Canada,CAN,1980Q1-2013Q4,136
7,Switzerland,CHE,1980Q1-2013Q4,136
8,Chile,CHL,1996Q1-2013Q4,72
9,Colombia,COL,2005Q1-2013Q4,36


In [249]:
# pcon

period_var_46_countries('pcon')

,country,isocode,period,obs
0,Argentina,ARG,2004Q1-2013Q4,40
1,Australia,AUS,1980Q1-2013Q4,136
2,Austria,AUT,1995Q1-2013Q4,76
3,Belgium,BEL,1995Q1-2013Q4,76
4,Bulgaria,BGR,1999Q1-2013Q4,60
5,Brazil,BRA,1996Q1-2013Q4,72
6,Canada,CAN,1981Q1-2013Q4,132
7,Switzerland,CHE,1980Q1-2013Q4,136
8,Chile,CHL,1996Q1-2013Q4,72
9,Colombia,COL,2005Q1-2013Q4,36


In [250]:
# reer

period_var_46_countries('reer')

,country,isocode,period,obs
0,Argentina,ARG,1994Q1-2013Q4,80
1,Australia,AUS,1980Q1-2013Q4,136
2,Austria,AUT,1979Q1-2013Q4,140
3,Belgium,BEL,1991Q1-2013Q4,92
4,Bulgaria,BGR,1999Q1-2013Q4,60
5,Brazil,BRA,1980Q1-2013Q4,136
6,Canada,CAN,1980Q1-2013Q4,136
7,Switzerland,CHE,1979Q1-2013Q4,140
8,Chile,CHL,1989Q1-2013Q4,100
9,Colombia,COL,1980Q1-2013Q4,136


In [251]:
# ca

period_var_46_countries('ca')

,country,isocode,period,obs
0,Argentina,ARG,2004Q1-2013Q4,40
1,Australia,AUS,1980Q1-2013Q4,136
2,Austria,AUT,1984Q1-2013Q4,120
3,Belgium,BEL,1991Q1-2013Q4,92
4,Bulgaria,BGR,1999Q1-2013Q4,60
5,Brazil,BRA,1996Q1-2013Q4,72
6,Canada,CAN,1980Q1-2013Q4,136
7,Switzerland,CHE,1985Q1-2013Q4,116
8,Chile,CHL,1996Q1-2013Q4,72
9,Colombia,COL,2005Q1-2013Q4,36


In [252]:
# inv

period_var_46_countries('inv')

,country,isocode,period,obs
0,Argentina,ARG,2004Q1-2013Q4,40
1,Australia,AUS,1959Q3-2013Q4,218
2,Austria,AUT,1995Q1-2013Q4,76
3,Belgium,BEL,1995Q1-2013Q4,76
4,Bulgaria,BGR,1995Q1-2013Q4,76
5,Brazil,BRA,1996Q1-2013Q4,72
6,Canada,CAN,1961Q1-2013Q4,212
7,Switzerland,CHE,1980Q1-2013Q4,136
8,Chile,CHL,1996Q1-2013Q4,72
9,Colombia,COL,2005Q1-2013Q4,36


# **3. df_JME**

data source : https://www.ilzetzki.com/

In [253]:
# import the data

df_JME = pd.read_excel('df_JME.xlsx').sort_values(by=['country', 'quarter']).reset_index(drop=True)
df_wb_isocode = pd.read_csv('df_wb_isocode.csv', encoding='utf-8').rename(columns={'alpha-3':'isocode'})

In [254]:
# replace the country

df_JME = df_JME.replace({'country':'ARGENTINA'}, 'Argentina')
df_JME = df_JME.replace({'country':'AUSTRALIA'}, 'Australia')
df_JME = df_JME.replace({'country':'BELGIUM'}, 'Belgium')
df_JME = df_JME.replace({'country':'BOTSWANA'}, 'Botswana')
df_JME = df_JME.replace({'country':'BRAZIL'}, 'Brazil')
df_JME = df_JME.replace({'country':'BULGARIA'}, 'Bulgaria')
df_JME = df_JME.replace({'country':'CANADA'}, 'Canada')
df_JME = df_JME.replace({'country':'CHILE'}, 'Chile')
df_JME = df_JME.replace({'country':'COLOMBIA'}, 'Colombia')
df_JME = df_JME.replace({'country':'CROATIA'}, 'Croatia')
df_JME = df_JME.replace({'country':'CZECH'}, 'Czech Republic')
df_JME = df_JME.replace({'country':'DENMARK'}, 'Denmark')
df_JME = df_JME.replace({'country':'ECUADOR'}, 'Ecuador')
df_JME = df_JME.replace({'country':'EL_SALVADOR'}, 'El Salvador')
df_JME = df_JME.replace({'country':'ESTONIA'}, 'Estonia')
df_JME = df_JME.replace({'country':'FINLAND'}, 'Finland')
df_JME = df_JME.replace({'country':'FRANCE'}, 'France')
df_JME = df_JME.replace({'country':'GERMANY'}, 'Germany')
df_JME = df_JME.replace({'country':'GREECE'}, 'Greece')
df_JME = df_JME.replace({'country':'HUNGARY'}, 'Hungary')
df_JME = df_JME.replace({'country':'ICELAND'}, 'Iceland')
df_JME = df_JME.replace({'country':'IRELAND'}, 'Ireland')
df_JME = df_JME.replace({'country':'ISRAEL'}, 'Israel')
df_JME = df_JME.replace({'country':'ITALY'}, 'Italy')
df_JME = df_JME.replace({'country':'LATVIA'}, 'Latvia')
df_JME = df_JME.replace({'country':'LITHUANIA'}, 'Lithuania')
df_JME = df_JME.replace({'country':'MALAYSIA'}, 'Malaysia')
df_JME = df_JME.replace({'country':'MEXICO'}, 'Mexico')
df_JME = df_JME.replace({'country':'NETHERLANDS'}, 'Netherlands')
df_JME = df_JME.replace({'country':'NORWAY'}, 'Norway')
df_JME = df_JME.replace({'country':'PERU'}, 'Peru')
df_JME = df_JME.replace({'country':'POLAND'}, 'Poland')
df_JME = df_JME.replace({'country':'PORTUGAL'}, 'Portugal')
df_JME = df_JME.replace({'country':'ROMANIA'}, 'Romania')
df_JME = df_JME.replace({'country':'SLOVAK'}, 'Slovak Republic')
df_JME = df_JME.replace({'country':'SLOVENIA'}, 'Slovenia')
df_JME = df_JME.replace({'country':'SOUTH_AFRICA'}, 'South Africa')
df_JME = df_JME.replace({'country':'SPAIN'}, 'Spain')
df_JME = df_JME.replace({'country':'SWEDEN'}, 'Sweden')
df_JME = df_JME.replace({'country':'THAILAND'}, 'Thailand')
df_JME = df_JME.replace({'country':'TURKEY'}, 'Turkiye')
df_JME = df_JME.replace({'country':'UK'}, 'United Kingdom')
df_JME = df_JME.replace({'country':'URUGUAY'}, 'Uruguay')
df_JME = df_JME.replace({'country':'US'}, 'United States')

In [255]:
# merge df_JME with wb isocode data

df_JME = pd.merge(df_wb_isocode, df_JME, how='right', on=['country'])
df_JME = df_JME.sort_values(by=['isocode', 'quarter']).reset_index(drop=True)
df_JME = df_JME.rename(columns={'quarter':'time'})
df_JME['time'] = pd.to_datetime(df_JME['time'])
df_JME['time'] = df_JME['time'].dt.to_period('Q')

df_JME = df_JME[['country', 'isocode', 'time', 'ca', 'closed', 'ggdebt', 'dev', 'drate',
                 'fixed', 'fixeddum', 'flex', 'flexdum', 'ggcon', 'ggcongdp', 'gdp',
                 'gginv', 'gginvgdp', 'h', 'hdebt', 'htarf', 'large', 'ldebt', 'ltarf',
                 'open', 'pcon', 'pinv', 'reer', 'small', 'tariff']]

In [256]:
# merge df_JME with df_baseline

df_JME  = pd.merge(df_JME , df_baseline, how='right', on=['country', 'isocode', 'time']).drop(columns=['value'])

In [257]:
# df_JME

df_JME = df_JME.sort_values(by=['isocode', 'time']).reset_index(drop=True)
df_JME

,country,isocode,time,ca,closed,ggdebt,dev,drate,fixed,fixeddum,...,htarf,large,ldebt,ltarf,open,pcon,pinv,reer,small,tariff
0,Argentina,ARG,1950Q1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Argentina,ARG,1950Q2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Argentina,ARG,1950Q3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Argentina,ARG,1950Q4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Argentina,ARG,1951Q1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11771,South Africa,ZAF,2012Q4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11772,South Africa,ZAF,2013Q1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11773,South Africa,ZAF,2013Q2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11774,South Africa,ZAF,2013Q3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [258]:
# set the data

data = df_JME

In [259]:
# check the vars

df_JME.columns

Index(['country', 'isocode', 'time', 'ca', 'closed', 'ggdebt', 'dev', 'drate',
       'fixed', 'fixeddum', 'flex', 'flexdum', 'ggcon', 'ggcongdp', 'gdp',
       'gginv', 'gginvgdp', 'h', 'hdebt', 'htarf', 'large', 'ldebt', 'ltarf',
       'open', 'pcon', 'pinv', 'reer', 'small', 'tariff'],
      dtype='object')

In [260]:
# ggdebt

period_var_46_countries('ggcon')

,country,isocode,period,obs
0,Argentina,ARG,1993Q1-2006Q4,56
1,Australia,AUS,1960Q1-2008Q4,196
2,Austria,AUT,nan,nan
3,Belgium,BEL,1995Q1-2009Q4,60
4,Bulgaria,BGR,1999Q1-2008Q2,38
5,Brazil,BRA,1995Q1-2009Q4,60
6,Canada,CAN,1961Q1-2008Q4,192
7,Switzerland,CHE,nan,nan
8,Chile,CHL,1989Q1-2009Q3,83
9,Colombia,COL,1960Q1-2008Q4,59


# **4. df_OECD**

data source : https://stats.oecd.org/

In [261]:
"""
# debug HTTPConnectionPool error

from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter

retries = Retry(connect=5, read=3, redirect=3)
http_session = requests.Session()
http_session.mount('https://<yourdomain>.slack.com', HTTPAdapter(max_retries=retries))
"""

"\n# debug HTTPConnectionPool error\n\nfrom urllib3.util.retry import Retry\nfrom requests.adapters import HTTPAdapter\n\nretries = Retry(connect=5, read=3, redirect=3)\nhttp_session = requests.Session()\nhttp_session.mount('https://<yourdomain>.slack.com', HTTPAdapter(max_retries=retries))\n"

In [262]:
"""
# import the raw data object (EO: economic outlook)

start_time = dt.datetime(1950, 1, 1)
end_time = dt.datetime(2014, 1, 1)

df_oecd_eo_q_raw = pdr.oecd.OECDReader('EO', start=start_time, end=end_time, retry_count=3, pause=0.1, timeout=30, session=None, freq=None)
df_oecd_eo_q_raw = df_oecd_eo_q_raw.read()

df_oecd_eo_q_raw.index = pd.to_datetime(df_oecd_eo_q_raw.index)

print(type(df_oecd_eo_q_raw.index))
print(type(df_oecd_eo_q_raw.columns))
"""

"\n# import the raw data object (EO: economic outlook)\n\nstart_time = dt.datetime(1950, 1, 1)\nend_time = dt.datetime(2014, 1, 1)\n\ndf_oecd_eo_q_raw = pdr.oecd.OECDReader('EO', start=start_time, end=end_time, retry_count=3, pause=0.1, timeout=30, session=None, freq=None)\ndf_oecd_eo_q_raw = df_oecd_eo_q_raw.read()\n\ndf_oecd_eo_q_raw.index = pd.to_datetime(df_oecd_eo_q_raw.index)\n\nprint(type(df_oecd_eo_q_raw.index))\nprint(type(df_oecd_eo_q_raw.columns))\n"

In [263]:
"""
# import the raw data

var_name_list = ['Government final consumption expenditure, volume',
                 'Gross domestic product, volume, market prices',
                 'Gross domestic product, volume in USD, constant exchange rates',
                 'Current account balance as a percentage of GDP',
                 'Private final consumption expenditure, volume',
                 'Gross fixed capital formation, total, volume']

var_list = ['ggcon', 'gdp_dcu', 'gdp_usd', 'ca', 'pcon', 'inv']

for i in range(0, len(var_list)):

    globals()['df_{}'.format(var_list[i])] = df_oecd_eo_q_raw.xs(('Quarterly', var_name_list[i]),
                                                                 level=('Frequency', 'Variable'),
                                                                 axis=1)

    globals()['df_{}'.format(var_list[i])] = pd.DataFrame(globals()['df_{}'.format(var_list[i])].stack())
    globals()['df_{}'.format(var_list[i])] = globals()['df_{}'.format(var_list[i])].reset_index().rename(columns={'Country':'country', 'Time':'time', 0:var_list[i]})
    globals()['df_{}'.format(var_list[i])] = globals()['df_{}'.format(var_list[i])][['country', 'time', var_list[i]]]
    globals()['df_{}'.format(var_list[i])] = globals()['df_{}'.format(var_list[i])].sort_values(by=['country', 'time'])
    globals()['df_{}'.format(var_list[i])] = globals()['df_{}'.format(var_list[i])].reset_index().drop(columns=['index'])
"""

"\n# import the raw data\n\nvar_name_list = ['Government final consumption expenditure, volume',\n                 'Gross domestic product, volume, market prices',\n                 'Gross domestic product, volume in USD, constant exchange rates',\n                 'Current account balance as a percentage of GDP',\n                 'Private final consumption expenditure, volume',\n                 'Gross fixed capital formation, total, volume']\n\nvar_list = ['ggcon', 'gdp_dcu', 'gdp_usd', 'ca', 'pcon', 'inv']\n\nfor i in range(0, len(var_list)):\n\n    globals()['df_{}'.format(var_list[i])] = df_oecd_eo_q_raw.xs(('Quarterly', var_name_list[i]),\n                                                                 level=('Frequency', 'Variable'),\n                                                                 axis=1)\n\n    globals()['df_{}'.format(var_list[i])] = pd.DataFrame(globals()['df_{}'.format(var_list[i])].stack())\n    globals()['df_{}'.format(var_list[i])] = globals()['df_{}'.

In [264]:
"""
# merge the raw data

df_OECD = pd.merge(globals()['df_{}'.format(var_list[0])], globals()['df_{}'.format(var_list[1])], how='outer', on=['country', 'time'])

for var in var_list[2:]:

    df_OECD = pd.merge(df_OECD, globals()['df_{}'.format(var)], how='outer', on=['country', 'time'])
"""

"\n# merge the raw data\n\ndf_OECD = pd.merge(globals()['df_{}'.format(var_list[0])], globals()['df_{}'.format(var_list[1])], how='outer', on=['country', 'time'])\n\nfor var in var_list[2:]:\n\n    df_OECD = pd.merge(df_OECD, globals()['df_{}'.format(var)], how='outer', on=['country', 'time'])\n"

In [265]:
"""
# replace the country

# Korea, Rep.
df_OECD = df_OECD.replace({'country':'Korea'}, 'Korea, Rep.')

# China
df_OECD = df_OECD.replace({'country':"China (People's Republic of)"}, 'China')
"""

'\n# replace the country\n\n# Korea, Rep.\ndf_OECD = df_OECD.replace({\'country\':\'Korea\'}, \'Korea, Rep.\')\n\n# China\ndf_OECD = df_OECD.replace({\'country\':"China (People\'s Republic of)"}, \'China\')\n'

In [266]:
"""
# filter

# countries group
cond1 = (df_OECD['country'] == 'Euro area (17 countries)')
cond2 = (df_OECD['country'] == 'OECD - Total')

df_OECD = df_OECD.loc[~cond1 & ~cond2]
"""

"\n# filter\n\n# countries group\ncond1 = (df_OECD['country'] == 'Euro area (17 countries)')\ncond2 = (df_OECD['country'] == 'OECD - Total')\n\ndf_OECD = df_OECD.loc[~cond1 & ~cond2]\n"

In [267]:
"""
# merge the wb isocode data

df_wb_isocode = pd.read_csv('df_wb_isocode.csv', encoding='utf-8').rename(columns={'alpha-3':'isocode'})
df_OECD = pd.merge(df_wb_isocode, df_OECD, how='right', on=['country'])
df_OECD = df_OECD.sort_values(by=['isocode', 'time']).reset_index(drop=True)
df_OECD['time'] = pd.to_datetime(df_OECD['time'])
df_OECD['time'] = df_OECD['time'].dt.to_period('Q')
"""

"\n# merge the wb isocode data\n\ndf_wb_isocode = pd.read_csv('df_wb_isocode.csv', encoding='utf-8').rename(columns={'alpha-3':'isocode'})\ndf_OECD = pd.merge(df_wb_isocode, df_OECD, how='right', on=['country'])\ndf_OECD = df_OECD.sort_values(by=['isocode', 'time']).reset_index(drop=True)\ndf_OECD['time'] = pd.to_datetime(df_OECD['time'])\ndf_OECD['time'] = df_OECD['time'].dt.to_period('Q')\n"

In [268]:
"""
# select the T

cond_T = (df_OECD['time'].dt.year < 2014)

df_OECD = df_OECD.loc[cond_T]
"""

"\n# select the T\n\ncond_T = (df_OECD['time'].dt.year < 2014)\n\ndf_OECD = df_OECD.loc[cond_T]\n"

In [269]:
"""
# export the data

df_OECD.to_excel(excel_writer='df_OECD.xlsx')
"""

"\n# export the data\n\ndf_OECD.to_excel(excel_writer='df_OECD.xlsx')\n"

In [270]:
# import the data

df_OECD = pd.read_excel('df_OECD.xlsx').reset_index(drop=True).drop(columns={'Unnamed: 0'})
df_OECD['time'] = pd.to_datetime(df_OECD['time'])
df_OECD['time'] = df_OECD['time'].dt.to_period('Q')

In [271]:
# merge df_OECD with df_baseline

df_OECD  = pd.merge(df_OECD , df_baseline, how='right', on=['country', 'isocode', 'time']).drop(columns=['value'])

In [272]:
# df_OECD

df_OECD = df_OECD.sort_values(by=['isocode', 'time']).reset_index(drop=True)
df_OECD

,country,isocode,time,ggcon,gdp_dcu,gdp_usd,ca,pcon,inv
0,Argentina,ARG,1950Q1,NaN,NaN,NaN,NaN,NaN,NaN
1,Argentina,ARG,1950Q2,NaN,NaN,NaN,NaN,NaN,NaN
2,Argentina,ARG,1950Q3,NaN,NaN,NaN,NaN,NaN,NaN
3,Argentina,ARG,1950Q4,NaN,NaN,NaN,NaN,NaN,NaN
4,Argentina,ARG,1951Q1,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
11771,South Africa,ZAF,2012Q4,NaN,NaN,NaN,NaN,NaN,NaN
11772,South Africa,ZAF,2013Q1,NaN,NaN,NaN,NaN,NaN,NaN
11773,South Africa,ZAF,2013Q2,NaN,NaN,NaN,NaN,NaN,NaN
11774,South Africa,ZAF,2013Q3,NaN,NaN,NaN,NaN,NaN,NaN


# **5. df_WEO**

data source : https://www.imf.org/en/Publications/WEO/weo-database/2022/October/download-entire-database

In [273]:
# import the data

df_WEO = pd.read_excel('df_WEO.xlsx').reset_index(drop=True)

In [274]:
# replace the country

df_WEO = df_WEO.replace({'country':'Hong Kong SAR'}, 'Hong Kong SAR, China')
df_WEO = df_WEO.replace({'country':'Korea'}, 'Korea, Rep.')
df_WEO = df_WEO.replace({'country':'Russia'}, 'Russian Federation')
df_WEO = df_WEO.replace({'country':'T?kiye'}, 'Turkiye')

In [275]:
# edit the df_WEO

isocode_array = np.array(['ARG', 'AUS', 'AUT', 'BEL', 'BGR', 'BRA', 'CAN', 'CHE', 'CHL', 'COL',
                          'CZE', 'DEU', 'DNK', 'ESP', 'EST', 'FIN', 'FRA', 'GBR', 'GRC', 'HKG',
                          'HRV', 'HUN', 'IDN', 'IND', 'IRL', 'ISR', 'ITA', 'JPN', 'KOR', 'LTU',
                          'LVA', 'MEX', 'NLD', 'NOR', 'NZL', 'POL', 'PRT', 'ROU', 'RUS', 'SVK',
                          'SVN', 'SWE', 'THA', 'TUR', 'USA', 'ZAF'])

for i in range(0, isocode_array.shape[0]):

    globals()['cond_N{}'.format(i+1)] = (df_WEO['isocode'] == isocode_array[i])

df_WEO = df_WEO.loc[cond_N1 | cond_N2 | cond_N3 | cond_N4 | cond_N5 | cond_N6 | cond_N7 | cond_N8 | cond_N9 | cond_N10 |
                    cond_N11 | cond_N12 | cond_N13 | cond_N14 | cond_N15 | cond_N16 | cond_N17 | cond_N18 | cond_N19 | cond_N20 |
                    cond_N21 | cond_N22 | cond_N23 | cond_N24 | cond_N25 | cond_N26 | cond_N27 | cond_N28 | cond_N29 | cond_N30 |
                    cond_N31 | cond_N32 | cond_N33 | cond_N34 | cond_N35 | cond_N36 | cond_N37 | cond_N38 | cond_N39 | cond_N40 |
                    cond_N41 | cond_N42 | cond_N43 | cond_N44 | cond_N45 | cond_N46]

df_WEO = df_WEO.sort_values(by=['isocode']).reset_index(drop=True)
df_WEO

,isocode,country,Subject Descriptor,units,scale,1980,1981,1982,1983,1984,...,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027
0,ARG,Argentina,"Gross domestic product, constant prices",National currency,Billions,357.389,336.861,326.255,338.435,345.205,...,707.377,693.224,624.295,689.211,717.034,731.548,746.319,761.249,776.475,792.005
1,ARG,Argentina,Unemployment rate,Percent of total labor force,NaN,3,5,4.5,5,5,...,9.2,9.825,11.55,8.725,6.9,6.9,6.9,6.9,6.9,6.9
2,ARG,Argentina,Employment,Persons,Millions,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ARG,Argentina,Population,Persons,Millions,27.95,28.45,28.93,29.34,29.84,...,44.495,44.939,45.388,45.842,46.3,46.763,47.231,47.703,48.18,48.662
4,ARG,Argentina,General government revenue,National currency,Billions,NaN,NaN,NaN,NaN,NaN,...,4940.7,7261.81,9198.32,15506.69,26986.16,47528.48,74928.64,110270.77,157026.08,214969.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019,ZAF,South Africa,General government revenue,National currency,Billions,NaN,NaN,NaN,NaN,NaN,...,1416.96,1501.55,1387.35,1668.23,1846.4,1921.32,1975.67,2081.45,2210.91,2341.49
2020,ZAF,South Africa,Population,Persons,Millions,29.08,29.83,30.61,31.41,32.21,...,57.939,58.775,59.622,60.143,61.06,61.991,62.937,63.897,64.871,65.861
2021,ZAF,South Africa,Employment,Persons,Millions,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022,ZAF,South Africa,Volume of exports of goods,Percent change,NaN,0.624,-4.186,-2.612,-0.182,1.754,...,3.65,-3.538,-11.863,9.987,5.475,4.6,4.275,3.992,3.85,3.85


In [276]:
# list of subject descriptor

df_WEO['Subject Descriptor'].unique()

array(['Gross domestic product, constant prices', 'Unemployment rate',
       'Employment', 'Population', 'General government revenue',
       'General government total expenditure',
       'General government net lending/borrowing',
       'General government structural balance',
       'General government primary net lending/borrowing',
       'General government net debt', 'General government gross debt',
       'Gross domestic product corresponding to fiscal year, current prices',
       'Current account balance', 'Volume of exports of goods',
       'Volume of exports of goods and services',
       'Volume of imports of goods and services',
       'Volume of Imports of goods',
       'Gross domestic product, current prices',
       'Gross domestic product, deflator',
       'Gross domestic product per capita, constant prices',
       'Gross domestic product per capita, current prices',
       'Gross domestic product based on purchasing-power-parity (PPP) share of world total',
   

# **6. ggdebt**

## **6.1. df_WEO_ggdebt**

In [277]:
# generate df_WEO_ggdebt

cond_ggdebt = (df_WEO['Subject Descriptor'] == 'General government gross debt')
cond_units = (df_WEO['units'] == 'Percent of GDP')

df_WEO_ggdebt = df_WEO.loc[(cond_ggdebt & cond_units)].reset_index(drop=True)
df_WEO_ggdebt = df_WEO_ggdebt[['isocode', 'country',
                               1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989,
                               1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999,
                               2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009,
                               2010, 2011, 2012, 2013]]

df_WEO_ggdebt = df_WEO_ggdebt.set_index(['isocode', 'country']).transpose()
df_WEO_ggdebt = pd.DataFrame(df_WEO_ggdebt.stack(level=['isocode', 'country'])).reset_index()
df_WEO_ggdebt = df_WEO_ggdebt.rename(columns={'level_0':'year', 0:'ggdebt'})
df_WEO_ggdebt = df_WEO_ggdebt[['country', 'isocode', 'year', 'ggdebt']].sort_values(by=['isocode', 'year']).reset_index(drop=True)

In [278]:
# generate the q1 dataset

# set the data
df_WEO_q1 = df_WEO_ggdebt

# q1 function transforming y type into ymd type
def ymd_q1(x):

    # numerical type, not string type -> indexing is not applied
    result = str(x)
    return result[0:4] + '-01-01'

# transform year into quarter
df_WEO_q1['year'] = pd.DataFrame(df_WEO_q1['year'].apply(ymd_q1))
df_WEO_q1['year'] = pd.to_datetime(df_WEO_q1['year'])
df_WEO_q1['year'] = df_WEO_q1['year'].dt.to_period('Q')
df_WEO_q1 = df_WEO_q1.rename(columns={'year':'quarter'})
df_WEO_q1 = df_WEO_q1.sort_values(by=['isocode', 'quarter']).reset_index(drop=True)

In [279]:
# generate the q2 dataset

# set the data
df_WEO_q2 = df_WEO_ggdebt

# q2 function transforming y type into ymd type
def ymd_q2(x):

    # numerical type, not string type -> indexing is not applied
    result = str(x)
    return result[0:4] + '-04-01'

# transform year into quarter
df_WEO_q2['year'] = pd.DataFrame(df_WEO_q2['year'].apply(ymd_q2))
df_WEO_q2['year'] = pd.to_datetime(df_WEO_q2['year'])
df_WEO_q2['year'] = df_WEO_q2['year'].dt.to_period('Q')
df_WEO_q2 = df_WEO_q2.rename(columns={'year':'quarter'})
df_WEO_q2 = df_WEO_q2.sort_values(by=['isocode', 'quarter']).reset_index(drop=True)

In [280]:
# generate the q3 dataset

# set the data
df_WEO_q3 = df_WEO_ggdebt

# q3 function transforming y type into ymd type
def ymd_q3(x):

    # numerical type, not string type -> indexing is not applied
    result = str(x)
    return result[0:4] + '-07-01'

# transform year into quarter
df_WEO_q3['year'] = pd.DataFrame(df_WEO_q3['year'].apply(ymd_q3))
df_WEO_q3['year'] = pd.to_datetime(df_WEO_q3['year'])
df_WEO_q3['year'] = df_WEO_q3['year'].dt.to_period('Q')
df_WEO_q3 = df_WEO_q3.rename(columns={'year':'quarter'})
df_WEO_q3 = df_WEO_q3.sort_values(by=['isocode', 'quarter']).reset_index(drop=True)

In [281]:
# generate the q4 dataset

# set the data
df_WEO_q4 = df_WEO_ggdebt

# q4 function transforming y type into ymd type
def ymd_q4(x):

    # numerical type, not string type -> indexing is not applied
    result = str(x)
    return result[0:4] + '-12-01'

# transform year into quarter
df_WEO_q4['year'] = pd.DataFrame(df_WEO_q4['year'].apply(ymd_q4))
df_WEO_q4['year'] = pd.to_datetime(df_WEO_q4['year'])
df_WEO_q4['year'] = df_WEO_q4['year'].dt.to_period('Q')
df_WEO_q4 = df_WEO_q4.rename(columns={'year':'quarter'})
df_WEO_q4 = df_WEO_q4.sort_values(by=['isocode', 'quarter']).reset_index(drop=True)

In [282]:
# merge the raw data

df_WEO_ggdebt = pd.merge(df_WEO_q1, df_WEO_q2, how='outer', on=['country', 'isocode', 'quarter', 'ggdebt'])
df_WEO_ggdebt = pd.merge(df_WEO_ggdebt, df_WEO_q3, how='outer', on=['country', 'isocode', 'quarter', 'ggdebt'])
df_WEO_ggdebt = pd.merge(df_WEO_ggdebt, df_WEO_q4, how='outer', on=['country', 'isocode', 'quarter', 'ggdebt'])

<ipython-input-282-ce605eed7840>:3: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  df_WEO_ggdebt = pd.merge(df_WEO_q1, df_WEO_q2, how='outer', on=['country', 'isocode', 'quarter', 'ggdebt'])
<ipython-input-282-ce605eed7840>:4: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  df_WEO_ggdebt = pd.merge(df_WEO_ggdebt, df_WEO_q3, how='outer', on=['country', 'isocode', 'quarter', 'ggdebt'])
<ipython-input-282-ce605eed7840>:5: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  df_WEO_ggdebt = pd.merge(df_WEO_ggdebt, df_WEO_q4, how='outer', on=['country', 'isocode', 'quarter', 'ggdebt'])


In [283]:
# sort the data by year

df_WEO_ggdebt['year'] = df_WEO_ggdebt['quarter'].dt.year
df_WEO_ggdebt = df_WEO_ggdebt.sort_values(by=['isocode', 'year']).reset_index(drop=True)
df_WEO_ggdebt = df_WEO_ggdebt.drop(columns=['year']).reset_index(drop=True)

In [284]:
# select the T and N

# select the T
cond_T = (df_WEO_ggdebt['quarter'].dt.year >= 1980)

df_WEO_ggdebt = df_WEO_ggdebt.loc[cond_T]

# select the N
isocode_array = np.array(['ARG', 'AUS', 'AUT', 'BEL', 'BGR', 'BRA', 'CAN', 'CHE', 'CHL', 'COL',
                          'CZE', 'DEU', 'DNK', 'ESP', 'EST', 'FIN', 'FRA', 'GBR', 'GRC', 'HKG',
                          'HRV', 'HUN', 'IDN', 'IND', 'IRL', 'ISR', 'ITA', 'JPN', 'KOR', 'LTU',
                          'LVA', 'MEX', 'NLD', 'NOR', 'NZL', 'POL', 'PRT', 'ROU', 'RUS', 'SVK',
                          'SVN', 'SWE', 'THA', 'TUR', 'USA', 'ZAF'])

for i in range(0, isocode_array.shape[0]):

    globals()['cond_N{}'.format(i+1)] = (df_WEO_ggdebt['isocode'] == isocode_array[i])

df_WEO_ggdebt = df_WEO_ggdebt.loc[cond_N1 | cond_N2 | cond_N3 | cond_N4 | cond_N5 | cond_N6 | cond_N7 | cond_N8 | cond_N9 | cond_N10 |
                                  cond_N11 | cond_N12 | cond_N13 | cond_N14 | cond_N15 | cond_N16 | cond_N17 | cond_N18 | cond_N19 | cond_N20 |
                                  cond_N21 | cond_N22 | cond_N23 | cond_N24 | cond_N25 | cond_N26 | cond_N27 | cond_N28 | cond_N29 | cond_N30 |
                                  cond_N31 | cond_N32 | cond_N33 | cond_N34 | cond_N35 | cond_N36 | cond_N37 | cond_N38 | cond_N39 | cond_N40 |
                                  cond_N41 | cond_N42 | cond_N43 | cond_N44 | cond_N45 | cond_N46]

In [285]:
# df_WEO_ggdebt

df_WEO_ggdebt = df_WEO_ggdebt.rename(columns={'quarter':'time'})
df_WEO_ggdebt = df_WEO_ggdebt.sort_values(by=['isocode', 'time']).reset_index(drop=True)
df_WEO_ggdebt = df_WEO_ggdebt.astype({'ggdebt':'float'})

## **6.2. df_JME_WEO_ggdebt**

In [286]:
# generate the df_JME_WEO

df_JME_ggdebt = df_JME[['country', 'isocode', 'time', 'ggdebt']].rename(columns={'ggdebt':'JME_ggdebt'})
df_WEO_ggdebt = df_WEO_ggdebt[['country', 'isocode', 'time', 'ggdebt']].rename(columns={'ggdebt':'WEO_ggdebt'})

df_JME_WEO_ggdebt = pd.merge(df_JME_ggdebt, df_WEO_ggdebt, how='outer', on=['country', 'isocode', 'time'])
df_JME_WEO_ggdebt['ggdebt'] = df_JME_WEO_ggdebt['JME_ggdebt']

df_JME_WEO_ggdebt['ggdebt'] = df_JME_WEO_ggdebt['ggdebt'].fillna(df_JME_WEO_ggdebt['WEO_ggdebt'])

df_JME_WEO_ggdebt = df_JME_WEO_ggdebt[['country', 'isocode', 'time', 'ggdebt']]
df_JME_WEO_ggdebt

,country,isocode,time,ggdebt
0,Argentina,ARG,1950Q1,NaN
1,Argentina,ARG,1950Q2,NaN
2,Argentina,ARG,1950Q3,NaN
3,Argentina,ARG,1950Q4,NaN
4,Argentina,ARG,1951Q1,NaN
...,...,...,...,...
11771,South Africa,ZAF,2012Q4,37.408
11772,South Africa,ZAF,2013Q1,40.356
11773,South Africa,ZAF,2013Q2,40.356
11774,South Africa,ZAF,2013Q3,40.356


In [287]:
# merge df_JME_WEO_ggdebt with df_baseline

df_JME_WEO_ggdebt  = pd.merge(df_JME_WEO_ggdebt , df_baseline, how='right', on=['country', 'isocode', 'time']).drop(columns=['value'])

In [288]:
# df_JME_WEO_ggdebt

df_JME_WEO_ggdebt = df_JME_WEO_ggdebt.sort_values(by=['isocode', 'time']).reset_index(drop=True)
df_JME_WEO_ggdebt = df_JME_WEO_ggdebt.rename(columns={'ggdebt':'JME_WEO_ggdebt'})
df_JME_WEO_ggdebt

,country,isocode,time,JME_WEO_ggdebt
0,Argentina,ARG,1950Q1,NaN
1,Argentina,ARG,1950Q2,NaN
2,Argentina,ARG,1950Q3,NaN
3,Argentina,ARG,1950Q4,NaN
4,Argentina,ARG,1951Q1,NaN
...,...,...,...,...
11771,South Africa,ZAF,2012Q4,37.408
11772,South Africa,ZAF,2013Q1,40.356
11773,South Africa,ZAF,2013Q2,40.356
11774,South Africa,ZAF,2013Q3,40.356


In [289]:
# set the data

data = df_JME_WEO_ggdebt

In [290]:
# JME_WEO_ggdebt

period_var_46_countries('JME_WEO_ggdebt')

# list of missing sample period (JME_WEO_ggdebt)
# CHL : 1989:Q1 - 1990:Q4
# LTU : 1995:Q1 - 1997:Q4

,country,isocode,period,obs
0,Argentina,ARG,1990Q1-2013Q4,96
1,Australia,AUS,1980Q1-2013Q4,136
2,Austria,AUT,1988Q1-2013Q4,104
3,Belgium,BEL,1980Q1-2013Q4,136
4,Bulgaria,BGR,1991Q1-2013Q4,92
5,Brazil,BRA,1991Q1-2013Q4,92
6,Canada,CAN,1980Q1-2013Q4,136
7,Switzerland,CHE,1990Q1-2013Q4,96
8,Chile,CHL,1991Q1-2013Q4,92
9,Colombia,COL,1990Q1-2013Q4,96


# **7. ggcon**

## **7.1. df_OECD_ggcon**

In [291]:
# generate df_OECD_ggcon

df_OECD_ggcon = df_OECD[['country', 'isocode', 'time', 'ggcon']]

In [292]:
# seasonal adjustment

df_temp = df_OECD_ggcon

for i in tqdm(range(0, df_temp['isocode'].unique().shape[0])):

    globals()['cond_isocode{}'.format(i+1)] = (df_temp['isocode'] == df_temp['isocode'].unique()[i])
    globals()['df_temp_isocode{}'.format(i+1)] = df_temp.loc[globals()['cond_isocode{}'.format(i+1)]]

    globals()['cond_null{}'.format(i+1)] = (globals()['df_temp_isocode{}'.format(i+1)]['ggcon'].isnull() == True)

    globals()['df_temp_sa{}'.format(i+1)] = globals()['df_temp_isocode{}'.format(i+1)].loc[(globals()['cond_isocode{}'.format(i+1)] & ~globals()['cond_null{}'.format(i+1)])]

    df_temp.loc[(globals()['cond_isocode{}'.format(i+1)] & ~globals()['cond_null{}'.format(i+1)]), 'ggcon'] = stld(globals()['df_temp_sa{}'.format(i+1)]['ggcon'])

df_OECD_ggcon = df_temp

100%|██████████| 46/46 [00:00<00:00, 103.93it/s]


In [293]:
# merge df_OECD_ggcon with df_baseline

df_OECD_ggcon  = pd.merge(df_OECD_ggcon, df_baseline, how='right', on=['country', 'isocode', 'time']).drop(columns=['value'])

In [294]:
# df_OECD_ggcon

df_OECD_ggcon = df_OECD_ggcon.sort_values(by=['isocode', 'time']).reset_index(drop=True)
df_OECD_ggcon = df_OECD_ggcon.rename(columns={'ggcon':'OECD_ggcon'})
df_OECD_ggcon

,country,isocode,time,OECD_ggcon
0,Argentina,ARG,1950Q1,NaN
1,Argentina,ARG,1950Q2,NaN
2,Argentina,ARG,1950Q3,NaN
3,Argentina,ARG,1950Q4,NaN
4,Argentina,ARG,1951Q1,NaN
...,...,...,...,...
11771,South Africa,ZAF,2012Q4,NaN
11772,South Africa,ZAF,2013Q1,NaN
11773,South Africa,ZAF,2013Q2,NaN
11774,South Africa,ZAF,2013Q3,NaN


In [295]:
# set the data

data = df_OECD_ggcon

In [296]:
# OECD ggcon

period_var_46_countries('OECD_ggcon')

# list of replacement
# BEL : 1976:Q2-2013:Q4
# CAN : 1976:Q2-2013:Q4
# GBR : 1976:Q2-2013:Q4
# NLD : 1976:Q2-2013:Q4
# PRT : 1976:Q2-2013:Q4

,country,isocode,period,obs
0,Argentina,ARG,nan,nan
1,Australia,AUS,1976Q2-2013Q4,151
2,Austria,AUT,1984Q2-2013Q4,119
3,Belgium,BEL,1976Q2-2013Q4,151
4,Bulgaria,BGR,nan,nan
5,Brazil,BRA,nan,nan
6,Canada,CAN,1976Q2-2013Q4,151
7,Switzerland,CHE,1979Q2-2013Q4,139
8,Chile,CHL,2010Q2-2013Q4,15
9,Colombia,COL,2008Q2-2013Q4,23


## **7.2. df_TUR_ggcon**

data source : https://data.tuik.gov.tr/Kategori/GetKategori?p=ulusal-hesaplar-113&dil=2

In [297]:
# import the data

df_TUR = pd.read_excel('df_TUR.xlsx').reset_index(drop=True)
df_TUR['time'] = pd.to_datetime(df_TUR['time'])
df_TUR['time'] = df_TUR['time'].dt.to_period('Q')

In [298]:
# seasonal adjustment

df_temp = df_TUR

for i in tqdm(range(0, df_temp['isocode'].unique().shape[0])):

    globals()['cond_isocode{}'.format(i+1)] = (df_temp['isocode'] == df_temp['isocode'].unique()[i])
    globals()['df_temp_isocode{}'.format(i+1)] = df_temp.loc[globals()['cond_isocode{}'.format(i+1)]]

    # ggcon
    globals()['cond_null{}'.format(i+1)] = (globals()['df_temp_isocode{}'.format(i+1)]['ggcon'].isnull() == True)
    globals()['df_temp_sa{}'.format(i+1)] = globals()['df_temp_isocode{}'.format(i+1)].loc[(globals()['cond_isocode{}'.format(i+1)] & ~globals()['cond_null{}'.format(i+1)])]
    df_temp.loc[(globals()['cond_isocode{}'.format(i+1)] & ~globals()['cond_null{}'.format(i+1)]), 'ggcon'] = stld(globals()['df_temp_sa{}'.format(i+1)]['ggcon'])

    # pcon
    globals()['cond_null{}'.format(i+1)] = (globals()['df_temp_isocode{}'.format(i+1)]['pcon'].isnull() == True)
    globals()['df_temp_sa{}'.format(i+1)] = globals()['df_temp_isocode{}'.format(i+1)].loc[(globals()['cond_isocode{}'.format(i+1)] & ~globals()['cond_null{}'.format(i+1)])]
    df_temp.loc[(globals()['cond_isocode{}'.format(i+1)] & ~globals()['cond_null{}'.format(i+1)]), 'pcon'] = stld(globals()['df_temp_sa{}'.format(i+1)]['pcon'])

    # gdp
    globals()['cond_null{}'.format(i+1)] = (globals()['df_temp_isocode{}'.format(i+1)]['gdp'].isnull() == True)
    globals()['df_temp_sa{}'.format(i+1)] = globals()['df_temp_isocode{}'.format(i+1)].loc[(globals()['cond_isocode{}'.format(i+1)] & ~globals()['cond_null{}'.format(i+1)])]
    df_temp.loc[(globals()['cond_isocode{}'.format(i+1)] & ~globals()['cond_null{}'.format(i+1)]), 'gdp'] = stld(globals()['df_temp_sa{}'.format(i+1)]['gdp'])

    # inv
    globals()['cond_null{}'.format(i+1)] = (globals()['df_temp_isocode{}'.format(i+1)]['inv'].isnull() == True)
    globals()['df_temp_sa{}'.format(i+1)] = globals()['df_temp_isocode{}'.format(i+1)].loc[(globals()['cond_isocode{}'.format(i+1)] & ~globals()['cond_null{}'.format(i+1)])]
    df_temp.loc[(globals()['cond_isocode{}'.format(i+1)] & ~globals()['cond_null{}'.format(i+1)]), 'inv'] = stld(globals()['df_temp_sa{}'.format(i+1)]['inv'])

df_TUR = df_temp

100%|██████████| 1/1 [00:00<00:00, 104.50it/s]


In [299]:
# merge df_TUR with df_baseline

df_TUR = pd.merge(df_TUR, df_baseline, how='right', on=['country', 'isocode', 'time']).drop(columns=['value'])

In [300]:
# df_TUR

df_TUR = df_TUR.sort_values(by=['isocode', 'time']).reset_index(drop=True)
df_TUR = df_TUR.rename(columns={'ggcon':'TUR_ggcon', 'pcon':'TUR_pcon', 'gdp':'TUR_gdp', 'inv':'TUR_inv'})
df_TUR

,country,isocode,time,TUR_ggcon,TUR_pcon,TUR_gdp,TUR_inv
0,Argentina,ARG,1950Q1,NaN,NaN,NaN,NaN
1,Argentina,ARG,1950Q2,NaN,NaN,NaN,NaN
2,Argentina,ARG,1950Q3,NaN,NaN,NaN,NaN
3,Argentina,ARG,1950Q4,NaN,NaN,NaN,NaN
4,Argentina,ARG,1951Q1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
11771,South Africa,ZAF,2012Q4,NaN,NaN,NaN,NaN
11772,South Africa,ZAF,2013Q1,NaN,NaN,NaN,NaN
11773,South Africa,ZAF,2013Q2,NaN,NaN,NaN,NaN
11774,South Africa,ZAF,2013Q3,NaN,NaN,NaN,NaN


In [301]:
# set the data

data = df_TUR

In [302]:
# TUR pcon

period_var_46_countries('TUR_pcon')

# list of replacement
# TUR : 1998:Q1-2008:Q4

,country,isocode,period,obs
0,Argentina,ARG,nan,nan
1,Australia,AUS,nan,nan
2,Austria,AUT,nan,nan
3,Belgium,BEL,nan,nan
4,Bulgaria,BGR,nan,nan
5,Brazil,BRA,nan,nan
6,Canada,CAN,nan,nan
7,Switzerland,CHE,nan,nan
8,Chile,CHL,nan,nan
9,Colombia,COL,nan,nan


# **8. gdp**

## **7.1. df_OECD_gdp**

In [303]:
# generate df_OECD_gdp

df_OECD_gdp = df_OECD[['country', 'isocode', 'time', 'gdp_dcu', 'gdp_usd']]

In [304]:
# seasonal adjustment

df_temp = df_OECD_gdp

for i in tqdm(range(0, df_temp['isocode'].unique().shape[0])):

    globals()['cond_isocode{}'.format(i+1)] = (df_temp['isocode'] == df_temp['isocode'].unique()[i])
    globals()['df_temp_isocode{}'.format(i+1)] = df_temp.loc[globals()['cond_isocode{}'.format(i+1)]]

    globals()['cond_null{}'.format(i+1)] = (globals()['df_temp_isocode{}'.format(i+1)]['gdp_dcu'].isnull() == True)

    globals()['df_temp_sa{}'.format(i+1)] = globals()['df_temp_isocode{}'.format(i+1)].loc[(globals()['cond_isocode{}'.format(i+1)] & ~globals()['cond_null{}'.format(i+1)])]

    df_temp.loc[(globals()['cond_isocode{}'.format(i+1)] & ~globals()['cond_null{}'.format(i+1)]), 'gdp_dcu'] = stld(globals()['df_temp_sa{}'.format(i+1)]['gdp_dcu'])

df_OECD_gdp = df_temp

100%|██████████| 46/46 [00:00<00:00, 105.41it/s]


In [305]:
# merge df_OECD_gdp with df_baseline

df_OECD_gdp  = pd.merge(df_OECD_gdp, df_baseline, how='right', on=['country', 'isocode', 'time']).drop(columns=['value'])

In [306]:
# df_OECD_gdp

df_OECD_gdp = df_OECD_gdp.sort_values(by=['isocode', 'time']).reset_index(drop=True)
df_OECD_gdp = df_OECD_gdp.rename(columns={'gdp_dcu':'OECD_gdp'})
df_OECD_gdp

,country,isocode,time,OECD_gdp,gdp_usd
0,Argentina,ARG,1950Q1,NaN,NaN
1,Argentina,ARG,1950Q2,NaN,NaN
2,Argentina,ARG,1950Q3,NaN,NaN
3,Argentina,ARG,1950Q4,NaN,NaN
4,Argentina,ARG,1951Q1,NaN,NaN
...,...,...,...,...,...
11771,South Africa,ZAF,2012Q4,NaN,NaN
11772,South Africa,ZAF,2013Q1,NaN,NaN
11773,South Africa,ZAF,2013Q2,NaN,NaN
11774,South Africa,ZAF,2013Q3,NaN,NaN


In [307]:
# set the data

data = df_OECD_gdp

In [308]:
# OECD gdp

period_var_46_countries('OECD_gdp')

# list of replacement
# BEL : 1976:Q2-2013:Q4
# CAN : 1976:Q2-2013:Q4
# GBR : 1976:Q2-2013:Q4
# NLD : 1976:Q2-2013:Q4
# PRT : 1976:Q2-2013:Q4

,country,isocode,period,obs
0,Argentina,ARG,nan,nan
1,Australia,AUS,1976Q2-2013Q4,151
2,Austria,AUT,1984Q2-2013Q4,119
3,Belgium,BEL,1976Q2-2013Q4,151
4,Bulgaria,BGR,nan,nan
5,Brazil,BRA,nan,nan
6,Canada,CAN,1976Q2-2013Q4,151
7,Switzerland,CHE,1979Q2-2013Q4,139
8,Chile,CHL,2000Q2-2013Q4,55
9,Colombia,COL,1989Q2-2013Q4,99


# **8. ca**

In [309]:
"""
# import the data

df_OECD_ca = pd.read_csv('df_OECD_ca.csv').reset_index(drop=True)
"""

"\n# import the data\n\ndf_OECD_ca = pd.read_csv('df_OECD_ca.csv').reset_index(drop=True)\n"

In [310]:
"""
# filter

cond1 = (df_OECD_ca['isocode'] == 'EA19')
cond2 = (df_OECD_ca['isocode'] == 'OECD')
cond3 = (df_OECD_ca['isocode'] == 'G-7')
cond4 = (df_OECD_ca['isocode'] == 'EU27_2020')

df_OECD_ca.loc[(~cond1 & ~cond2 & ~cond3 & ~cond4)].reset_index(drop=True)
"""

"\n# filter\n\ncond1 = (df_OECD_ca['isocode'] == 'EA19')\ncond2 = (df_OECD_ca['isocode'] == 'OECD')\ncond3 = (df_OECD_ca['isocode'] == 'G-7')\ncond4 = (df_OECD_ca['isocode'] == 'EU27_2020')\n\ndf_OECD_ca.loc[(~cond1 & ~cond2 & ~cond3 & ~cond4)].reset_index(drop=True)\n"

In [311]:
"""
# generate time var

def quarter(x):
    result = str(x) # numerical type, not string type -> indexing is not applied"
    return result[5:7]

def q1(x):
    result = str(x) # numerical type, not string type -> indexing is not applied"
    return result[0:4] + '-01-01'
def q2(x):
    result = str(x) # numerical type, not string type -> indexing is not applied"
    return result[0:4] + '-04-01'
def q3(x):
    result = str(x) # numerical type, not string type -> indexing is not applied"
    return result[0:4] + '-07-01'
def q4(x):
    result = str(x) # numerical type, not string type -> indexing is not applied"
    return result[0:4] + '-10-01'

df_OECD_ca['quarter'] = pd.DataFrame(df_OECD_ca['time'].apply(quarter))

df_OECD_ca.loc[(df_OECD_ca['quarter']=='Q1'), 'time'] = df_OECD_ca['time'].apply(q1)
df_OECD_ca.loc[(df_OECD_ca['quarter']=='Q2'), 'time'] = df_OECD_ca['time'].apply(q2)
df_OECD_ca.loc[(df_OECD_ca['quarter']=='Q3'), 'time'] = df_OECD_ca['time'].apply(q3)
df_OECD_ca.loc[(df_OECD_ca['quarter']=='Q4'), 'time'] = df_OECD_ca['time'].apply(q4)

df_OECD_ca['time'] = pd.to_datetime(df_OECD_ca['time'])
df_OECD_ca['time'] = df_OECD_ca['time'].dt.to_period('Q')
df_OECD_ca = df_OECD_ca[['isocode', 'time', 'ca']].reset_index(drop=True)
"""

'\n# generate time var\n\ndef quarter(x):\n    result = str(x) # numerical type, not string type -> indexing is not applied"\n    return result[5:7]\n\ndef q1(x):\n    result = str(x) # numerical type, not string type -> indexing is not applied"\n    return result[0:4] + \'-01-01\'\ndef q2(x):\n    result = str(x) # numerical type, not string type -> indexing is not applied"\n    return result[0:4] + \'-04-01\'\ndef q3(x):\n    result = str(x) # numerical type, not string type -> indexing is not applied"\n    return result[0:4] + \'-07-01\'\ndef q4(x):\n    result = str(x) # numerical type, not string type -> indexing is not applied"\n    return result[0:4] + \'-10-01\'\n \ndf_OECD_ca[\'quarter\'] = pd.DataFrame(df_OECD_ca[\'time\'].apply(quarter))\n\ndf_OECD_ca.loc[(df_OECD_ca[\'quarter\']==\'Q1\'), \'time\'] = df_OECD_ca[\'time\'].apply(q1)\ndf_OECD_ca.loc[(df_OECD_ca[\'quarter\']==\'Q2\'), \'time\'] = df_OECD_ca[\'time\'].apply(q2)\ndf_OECD_ca.loc[(df_OECD_ca[\'quarter\']==\'Q3\'), 

In [312]:
"""
# merge the wb isocode data

df_wb_isocode = pd.read_csv('df_wb_isocode.csv', encoding='utf-8').rename(columns={'alpha-3':'isocode'})
df_OECD_ca = pd.merge(df_wb_isocode, df_OECD_ca, how='right', on=['isocode'])
df_OECD_ca = df_OECD_ca.sort_values(by=['isocode', 'time']).reset_index(drop=True)
"""

"\n# merge the wb isocode data\n\ndf_wb_isocode = pd.read_csv('df_wb_isocode.csv', encoding='utf-8').rename(columns={'alpha-3':'isocode'})\ndf_OECD_ca = pd.merge(df_wb_isocode, df_OECD_ca, how='right', on=['isocode'])\ndf_OECD_ca = df_OECD_ca.sort_values(by=['isocode', 'time']).reset_index(drop=True)\n"

In [313]:
"""
# select the T and N

# select the T
cond_T = (df_OECD_ca['time'].dt.year >= 1950)

df_OECD_ca = df_OECD_ca.loc[cond_T]

# select the N
isocode_array = np.array(['ARG', 'AUS', 'AUT', 'BEL', 'BGR', 'BRA', 'CAN', 'CHE', 'CHL', 'COL',
                          'CZE', 'DEU', 'DNK', 'ESP', 'EST', 'FIN', 'FRA', 'GBR', 'GRC', 'HKG',
                          'HRV', 'HUN', 'IDN', 'IND', 'IRL', 'ISR', 'ITA', 'JPN', 'KOR', 'LTU',
                          'LVA', 'MEX', 'NLD', 'NOR', 'NZL', 'POL', 'PRT', 'ROU', 'RUS', 'SVK',
                          'SVN', 'SWE', 'THA', 'TUR', 'USA', 'ZAF'])

for i in range(0, isocode_array.shape[0]):

    globals()['cond_N{}'.format(i+1)] = (df_OECD_ca['isocode'] == isocode_array[i])

df_OECD_ca = df_OECD_ca.loc[cond_N1 | cond_N2 | cond_N3 | cond_N4 | cond_N5 | cond_N6 | cond_N7 | cond_N8 | cond_N9 | cond_N10 |
                            cond_N11 | cond_N12 | cond_N13 | cond_N14 | cond_N15 | cond_N16 | cond_N17 | cond_N18 | cond_N19 | cond_N20 |
                            cond_N21 | cond_N22 | cond_N23 | cond_N24 | cond_N25 | cond_N26 | cond_N27 | cond_N28 | cond_N29 | cond_N30 |
                            cond_N31 | cond_N32 | cond_N33 | cond_N34 | cond_N35 | cond_N36 | cond_N37 | cond_N38 | cond_N39 | cond_N40 |
                            cond_N41 | cond_N42 | cond_N43 | cond_N44 | cond_N45 | cond_N46]
"""

"\n# select the T and N\n\n# select the T\ncond_T = (df_OECD_ca['time'].dt.year >= 1950)\n\ndf_OECD_ca = df_OECD_ca.loc[cond_T]\n\n# select the N\nisocode_array = np.array(['ARG', 'AUS', 'AUT', 'BEL', 'BGR', 'BRA', 'CAN', 'CHE', 'CHL', 'COL',\n                          'CZE', 'DEU', 'DNK', 'ESP', 'EST', 'FIN', 'FRA', 'GBR', 'GRC', 'HKG',\n                          'HRV', 'HUN', 'IDN', 'IND', 'IRL', 'ISR', 'ITA', 'JPN', 'KOR', 'LTU',\n                          'LVA', 'MEX', 'NLD', 'NOR', 'NZL', 'POL', 'PRT', 'ROU', 'RUS', 'SVK',\n                          'SVN', 'SWE', 'THA', 'TUR', 'USA', 'ZAF'])\n\nfor i in range(0, isocode_array.shape[0]):\n\n    globals()['cond_N{}'.format(i+1)] = (df_OECD_ca['isocode'] == isocode_array[i])\n\ndf_OECD_ca = df_OECD_ca.loc[cond_N1 | cond_N2 | cond_N3 | cond_N4 | cond_N5 | cond_N6 | cond_N7 | cond_N8 | cond_N9 | cond_N10 |\n                            cond_N11 | cond_N12 | cond_N13 | cond_N14 | cond_N15 | cond_N16 | cond_N17 | cond_N18 | cond_N19 | con

## **8.1. df_OECD_ca**

In [314]:
# generate df_OECD_ca

df_OECD_ca = df_OECD[['country', 'isocode', 'time', 'ca']]

In [315]:
# seasonal adjustment

df_temp = df_OECD_ca

for i in tqdm(range(0, df_temp['isocode'].unique().shape[0])):

    globals()['cond_isocode{}'.format(i+1)] = (df_temp['isocode'] == df_temp['isocode'].unique()[i])
    globals()['df_temp_isocode{}'.format(i+1)] = df_temp.loc[globals()['cond_isocode{}'.format(i+1)]]

    globals()['cond_null{}'.format(i+1)] = (globals()['df_temp_isocode{}'.format(i+1)]['ca'].isnull() == True)

    globals()['df_temp_sa{}'.format(i+1)] = globals()['df_temp_isocode{}'.format(i+1)].loc[(globals()['cond_isocode{}'.format(i+1)] & ~globals()['cond_null{}'.format(i+1)])]

    df_temp.loc[(globals()['cond_isocode{}'.format(i+1)] & ~globals()['cond_null{}'.format(i+1)]), 'ca'] = stld(globals()['df_temp_sa{}'.format(i+1)]['ca'])

df_OECD_ca = df_temp

100%|██████████| 46/46 [00:00<00:00, 113.94it/s]


In [316]:
# merge df_OECD_ca with df_baseline

df_OECD_ca  = pd.merge(df_OECD_ca, df_baseline, how='right', on=['country', 'isocode', 'time']).drop(columns=['value'])

In [317]:
# df_OECD_ca

df_OECD_ca = df_OECD_ca.sort_values(by=['isocode', 'time']).reset_index(drop=True)
df_OECD_ca = df_OECD_ca.rename(columns={'ca':'OECD_ca'})
df_OECD_ca

,country,isocode,time,OECD_ca
0,Argentina,ARG,1950Q1,NaN
1,Argentina,ARG,1950Q2,NaN
2,Argentina,ARG,1950Q3,NaN
3,Argentina,ARG,1950Q4,NaN
4,Argentina,ARG,1951Q1,NaN
...,...,...,...,...
11771,South Africa,ZAF,2012Q4,NaN
11772,South Africa,ZAF,2013Q1,NaN
11773,South Africa,ZAF,2013Q2,NaN
11774,South Africa,ZAF,2013Q3,NaN


In [318]:
# set the data

data = df_OECD_ca

In [319]:
# OECD ca

period_var_46_countries('OECD_ca')

# list of replacement
# BEL : 1976:Q2-2013:Q4
# CAN : 1976:Q2-2013:Q4
# GBR : 1976:Q2-2013:Q4
# NLD : 1976:Q2-2013:Q4
# PRT : 1976:Q2-2013:Q4

,country,isocode,period,obs
0,Argentina,ARG,nan,nan
1,Australia,AUS,1976Q2-2013Q4,151
2,Austria,AUT,1984Q2-2013Q4,119
3,Belgium,BEL,1989Q2-2013Q4,99
4,Bulgaria,BGR,nan,nan
5,Brazil,BRA,nan,nan
6,Canada,CAN,1976Q2-2013Q4,151
7,Switzerland,CHE,1986Q2-2013Q4,111
8,Chile,CHL,2010Q2-2013Q4,15
9,Colombia,COL,2010Q2-2013Q4,15


## **8.2. df_WEO_ca**

In [320]:
# generate df_WEO_ca

cond_ca = (df_WEO['Subject Descriptor'] == 'Current account balance')
cond_units = (df_WEO['units'] == 'Percent of GDP')

df_WEO_ca = df_WEO.loc[(cond_ca & cond_units)].reset_index(drop=True)
df_WEO_ca = df_WEO_ca[['isocode', 'country',
                       1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989,
                       1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999,
                       2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009,
                       2010, 2011, 2012, 2013]]

df_WEO_ca = df_WEO_ca.set_index(['isocode', 'country']).transpose()
df_WEO_ca = pd.DataFrame(df_WEO_ca.stack(level=['isocode', 'country'])).reset_index()
df_WEO_ca = df_WEO_ca.rename(columns={'level_0':'year', 0:'ca'})
df_WEO_ca = df_WEO_ca[['country', 'isocode', 'year', 'ca']].sort_values(by=['isocode', 'year']).reset_index(drop=True)

In [321]:
# generate the q1 dataset

# set the data
df_WEO_q1 = df_WEO_ca

# q1 function transforming y type into ymd type
def ymd_q1(x):

    # numerical type, not string type -> indexing is not applied
    result = str(x)
    return result[0:4] + '-01-01'

# transform year into quarter
df_WEO_q1['year'] = pd.DataFrame(df_WEO_q1['year'].apply(ymd_q1))
df_WEO_q1['year'] = pd.to_datetime(df_WEO_q1['year'])
df_WEO_q1['year'] = df_WEO_q1['year'].dt.to_period('Q')
df_WEO_q1 = df_WEO_q1.rename(columns={'year':'quarter'})
df_WEO_q1 = df_WEO_q1.sort_values(by=['isocode', 'quarter']).reset_index(drop=True)

In [322]:
# generate the q2 dataset

# set the data
df_WEO_q2 = df_WEO_ca

# q2 function transforming y type into ymd type
def ymd_q2(x):

    # numerical type, not string type -> indexing is not applied
    result = str(x)
    return result[0:4] + '-04-01'

# transform year into quarter
df_WEO_q2['year'] = pd.DataFrame(df_WEO_q2['year'].apply(ymd_q2))
df_WEO_q2['year'] = pd.to_datetime(df_WEO_q2['year'])
df_WEO_q2['year'] = df_WEO_q2['year'].dt.to_period('Q')
df_WEO_q2 = df_WEO_q2.rename(columns={'year':'quarter'})
df_WEO_q2 = df_WEO_q2.sort_values(by=['isocode', 'quarter']).reset_index(drop=True)

In [323]:
# generate the q3 dataset

# set the data
df_WEO_q3 = df_WEO_ca

# q3 function transforming y type into ymd type
def ymd_q3(x):

    # numerical type, not string type -> indexing is not applied
    result = str(x)
    return result[0:4] + '-07-01'

# transform year into quarter
df_WEO_q3['year'] = pd.DataFrame(df_WEO_q3['year'].apply(ymd_q3))
df_WEO_q3['year'] = pd.to_datetime(df_WEO_q3['year'])
df_WEO_q3['year'] = df_WEO_q3['year'].dt.to_period('Q')
df_WEO_q3 = df_WEO_q3.rename(columns={'year':'quarter'})
df_WEO_q3 = df_WEO_q3.sort_values(by=['isocode', 'quarter']).reset_index(drop=True)

In [324]:
# generate the q4 dataset

# set the data
df_WEO_q4 = df_WEO_ca

# q4 function transforming y type into ymd type
def ymd_q4(x):

    # numerical type, not string type -> indexing is not applied
    result = str(x)
    return result[0:4] + '-12-01'

# transform year into quarter
df_WEO_q4['year'] = pd.DataFrame(df_WEO_q4['year'].apply(ymd_q4))
df_WEO_q4['year'] = pd.to_datetime(df_WEO_q4['year'])
df_WEO_q4['year'] = df_WEO_q4['year'].dt.to_period('Q')
df_WEO_q4 = df_WEO_q4.rename(columns={'year':'quarter'})
df_WEO_q4 = df_WEO_q4.sort_values(by=['isocode', 'quarter']).reset_index(drop=True)

In [325]:
# merge the raw data

df_WEO_ca = pd.merge(df_WEO_q1, df_WEO_q2, how='outer', on=['country', 'isocode', 'quarter', 'ca'])
df_WEO_ca = pd.merge(df_WEO_ca, df_WEO_q3, how='outer', on=['country', 'isocode', 'quarter', 'ca'])
df_WEO_ca = pd.merge(df_WEO_ca, df_WEO_q4, how='outer', on=['country', 'isocode', 'quarter', 'ca'])

<ipython-input-325-60cc38233075>:3: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  df_WEO_ca = pd.merge(df_WEO_q1, df_WEO_q2, how='outer', on=['country', 'isocode', 'quarter', 'ca'])
<ipython-input-325-60cc38233075>:4: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  df_WEO_ca = pd.merge(df_WEO_ca, df_WEO_q3, how='outer', on=['country', 'isocode', 'quarter', 'ca'])
<ipython-input-325-60cc38233075>:5: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  df_WEO_ca = pd.merge(df_WEO_ca, df_WEO_q4, how='outer', on=['country', 'isocode', 'quarter', 'ca'])


In [326]:
# sort the data by year

df_WEO_ca['year'] = df_WEO_ca['quarter'].dt.year
df_WEO_ca = df_WEO_ca.sort_values(by=['isocode', 'year']).reset_index(drop=True)
df_WEO_ca = df_WEO_ca.drop(columns=['year']).reset_index(drop=True)

In [327]:
# select the T and N

# select the T
cond_T = (df_WEO_ca['quarter'].dt.year >= 1980)

df_WEO_ca = df_WEO_ca.loc[cond_T]

# select the N
isocode_array = np.array(['ARG', 'AUS', 'AUT', 'BEL', 'BGR', 'BRA', 'CAN', 'CHE', 'CHL', 'COL',
                          'CZE', 'DEU', 'DNK', 'ESP', 'EST', 'FIN', 'FRA', 'GBR', 'GRC', 'HKG',
                          'HRV', 'HUN', 'IDN', 'IND', 'IRL', 'ISR', 'ITA', 'JPN', 'KOR', 'LTU',
                          'LVA', 'MEX', 'NLD', 'NOR', 'NZL', 'POL', 'PRT', 'ROU', 'RUS', 'SVK',
                          'SVN', 'SWE', 'THA', 'TUR', 'USA', 'ZAF'])

for i in range(0, isocode_array.shape[0]):

    globals()['cond_N{}'.format(i+1)] = (df_WEO_ca['isocode'] == isocode_array[i])

df_WEO_ca = df_WEO_ca.loc[cond_N1 | cond_N2 | cond_N3 | cond_N4 | cond_N5 | cond_N6 | cond_N7 | cond_N8 | cond_N9 | cond_N10 |
                          cond_N11 | cond_N12 | cond_N13 | cond_N14 | cond_N15 | cond_N16 | cond_N17 | cond_N18 | cond_N19 | cond_N20 |
                          cond_N21 | cond_N22 | cond_N23 | cond_N24 | cond_N25 | cond_N26 | cond_N27 | cond_N28 | cond_N29 | cond_N30 |
                          cond_N31 | cond_N32 | cond_N33 | cond_N34 | cond_N35 | cond_N36 | cond_N37 | cond_N38 | cond_N39 | cond_N40 |
                          cond_N41 | cond_N42 | cond_N43 | cond_N44 | cond_N45 | cond_N46]

In [328]:
# df_WEO_ca

df_WEO_ca = df_WEO_ca.rename(columns={'quarter':'time'})
df_WEO_ca = df_WEO_ca.sort_values(by=['isocode', 'time']).reset_index(drop=True)
df_WEO_ca = df_WEO_ca.astype({'ca':'float'})

In [329]:
# merge df_WEO_ca with df_baseline

df_WEO_ca  = pd.merge(df_WEO_ca, df_baseline, how='right', on=['country', 'isocode', 'time']).drop(columns=['value'])

In [330]:
# df_WEO_ca

df_WEO_ca = df_WEO_ca.sort_values(by=['isocode', 'time']).reset_index(drop=True)
df_WEO_ca = df_WEO_ca.rename(columns={'ca':'WEO_ca'})
df_WEO_ca

,country,isocode,time,WEO_ca
0,Argentina,ARG,1950Q1,NaN
1,Argentina,ARG,1950Q2,NaN
2,Argentina,ARG,1950Q3,NaN
3,Argentina,ARG,1950Q4,NaN
4,Argentina,ARG,1951Q1,NaN
...,...,...,...,...
11771,South Africa,ZAF,2012Q4,-4.689
11772,South Africa,ZAF,2013Q1,-5.326
11773,South Africa,ZAF,2013Q2,-5.326
11774,South Africa,ZAF,2013Q3,-5.326


In [331]:
# set the data

data = df_WEO_ca

In [332]:
# WEO ca

period_var_46_countries('WEO_ca')

,country,isocode,period,obs
0,Argentina,ARG,1980Q1-2013Q4,136
1,Australia,AUS,1980Q1-2013Q4,136
2,Austria,AUT,1980Q1-2013Q4,136
3,Belgium,BEL,1980Q1-2013Q4,136
4,Bulgaria,BGR,1980Q1-2013Q4,136
5,Brazil,BRA,1980Q1-2013Q4,136
6,Canada,CAN,1980Q1-2013Q4,136
7,Switzerland,CHE,1980Q1-2013Q4,136
8,Chile,CHL,1980Q1-2013Q4,136
9,Colombia,COL,1980Q1-2013Q4,136


## **8.3. df_COL_ca**

data source : https://totoro.banrep.gov.co/estadisticas-economicas/faces/pages/charts/line.xhtml?facesRedirect=true

In [333]:
# import the data

df_COL_ca = pd.read_excel('df_COL_ca.xlsx').reset_index(drop=True)
df_COL_ca['time'] = pd.to_datetime(df_COL_ca['time'])
df_COL_ca['time'] = df_COL_ca['time'].dt.to_period('Q')

In [334]:
# seasonal adjustment

df_temp = df_COL_ca

for i in tqdm(range(0, df_temp['isocode'].unique().shape[0])):

    globals()['cond_isocode{}'.format(i+1)] = (df_temp['isocode'] == df_temp['isocode'].unique()[i])
    globals()['df_temp_isocode{}'.format(i+1)] = df_temp.loc[globals()['cond_isocode{}'.format(i+1)]]

    globals()['cond_null{}'.format(i+1)] = (globals()['df_temp_isocode{}'.format(i+1)]['ca'].isnull() == True)

    globals()['df_temp_sa{}'.format(i+1)] = globals()['df_temp_isocode{}'.format(i+1)].loc[(globals()['cond_isocode{}'.format(i+1)] & ~globals()['cond_null{}'.format(i+1)])]

    df_temp.loc[(globals()['cond_isocode{}'.format(i+1)] & ~globals()['cond_null{}'.format(i+1)]), 'ca'] = stld(globals()['df_temp_sa{}'.format(i+1)]['ca'])

df_COL_ca = df_temp

100%|██████████| 1/1 [00:00<00:00, 250.30it/s]


In [335]:
# merge df_COL_ca with df_baseline

df_COL_ca  = pd.merge(df_COL_ca, df_baseline, how='right', on=['country', 'isocode', 'time']).drop(columns=['value'])

In [336]:
# df_COL_ca

df_COL_ca = df_COL_ca.sort_values(by=['isocode', 'time']).reset_index(drop=True)
df_COL_ca = df_COL_ca.rename(columns={'ca':'COL_ca'})
df_COL_ca

,country,isocode,time,COL_ca
0,Argentina,ARG,1950Q1,NaN
1,Argentina,ARG,1950Q2,NaN
2,Argentina,ARG,1950Q3,NaN
3,Argentina,ARG,1950Q4,NaN
4,Argentina,ARG,1951Q1,NaN
...,...,...,...,...
11771,South Africa,ZAF,2012Q4,NaN
11772,South Africa,ZAF,2013Q1,NaN
11773,South Africa,ZAF,2013Q2,NaN
11774,South Africa,ZAF,2013Q3,NaN


In [337]:
# set the data

data = df_COL_ca

In [338]:
# COL ca

period_var_46_countries('COL_ca')

# list of replacement
# COL : 1996:Q1-2013:Q4

,country,isocode,period,obs
0,Argentina,ARG,nan,nan
1,Australia,AUS,nan,nan
2,Austria,AUT,nan,nan
3,Belgium,BEL,nan,nan
4,Bulgaria,BGR,nan,nan
5,Brazil,BRA,nan,nan
6,Canada,CAN,nan,nan
7,Switzerland,CHE,nan,nan
8,Chile,CHL,nan,nan
9,Colombia,COL,1996Q1-2013Q4,72


# **9. pcon**

## **9.1. df_OECD_pcon**

In [339]:
# generate df_OECD_pcon

df_OECD_pcon = df_OECD[['country', 'isocode', 'time', 'pcon']]

In [340]:
# seasonal adjustment

df_temp = df_OECD_pcon

for i in tqdm(range(0, df_temp['isocode'].unique().shape[0])):

    globals()['cond_isocode{}'.format(i+1)] = (df_temp['isocode'] == df_temp['isocode'].unique()[i])
    globals()['df_temp_isocode{}'.format(i+1)] = df_temp.loc[globals()['cond_isocode{}'.format(i+1)]]

    globals()['cond_null{}'.format(i+1)] = (globals()['df_temp_isocode{}'.format(i+1)]['pcon'].isnull() == True)

    globals()['df_temp_sa{}'.format(i+1)] = globals()['df_temp_isocode{}'.format(i+1)].loc[(globals()['cond_isocode{}'.format(i+1)] & ~globals()['cond_null{}'.format(i+1)])]

    df_temp.loc[(globals()['cond_isocode{}'.format(i+1)] & ~globals()['cond_null{}'.format(i+1)]), 'pcon'] = stld(globals()['df_temp_sa{}'.format(i+1)]['pcon'])

df_OECD_pcon = df_temp

100%|██████████| 46/46 [00:00<00:00, 107.01it/s]


In [341]:
# merge df_OECD_pcon with df_baseline

df_OECD_pcon  = pd.merge(df_OECD_pcon, df_baseline, how='right', on=['country', 'isocode', 'time']).drop(columns=['value'])

In [342]:
# df_OECD_pcon

df_OECD_pcon = df_OECD_pcon.sort_values(by=['isocode', 'time']).reset_index(drop=True)
df_OECD_pcon = df_OECD_pcon.rename(columns={'pcon':'OECD_pcon'})
df_OECD_pcon

,country,isocode,time,OECD_pcon
0,Argentina,ARG,1950Q1,NaN
1,Argentina,ARG,1950Q2,NaN
2,Argentina,ARG,1950Q3,NaN
3,Argentina,ARG,1950Q4,NaN
4,Argentina,ARG,1951Q1,NaN
...,...,...,...,...
11771,South Africa,ZAF,2012Q4,NaN
11772,South Africa,ZAF,2013Q1,NaN
11773,South Africa,ZAF,2013Q2,NaN
11774,South Africa,ZAF,2013Q3,NaN


In [343]:
# set the data

data = df_OECD_pcon

In [344]:
# OECD pcon

period_var_46_countries('OECD_pcon')

# list of replacement
# BEL : 1976:Q2-2013:Q4
# CAN : 1976:Q2-2013:Q4
# GBR : 1976:Q2-2013:Q4
# NLD : 1976:Q2-2013:Q4
# PRT : 1976:Q2-2013:Q4

,country,isocode,period,obs
0,Argentina,ARG,nan,nan
1,Australia,AUS,1976Q2-2013Q4,151
2,Austria,AUT,1984Q2-2013Q4,119
3,Belgium,BEL,1976Q2-2013Q4,151
4,Bulgaria,BGR,nan,nan
5,Brazil,BRA,nan,nan
6,Canada,CAN,1976Q2-2013Q4,151
7,Switzerland,CHE,1979Q2-2013Q4,139
8,Chile,CHL,2010Q2-2013Q4,15
9,Colombia,COL,2008Q2-2013Q4,23


# **10. inv**

## **10.1. df_OECD_inv**

In [345]:
# generate df_OECD_inv

df_OECD_inv = df_OECD[['country', 'isocode', 'time', 'inv']]

In [346]:
# seasonal adjustment

df_temp = df_OECD_inv

for i in tqdm(range(0, df_temp['isocode'].unique().shape[0])):

    globals()['cond_isocode{}'.format(i+1)] = (df_temp['isocode'] == df_temp['isocode'].unique()[i])
    globals()['df_temp_isocode{}'.format(i+1)] = df_temp.loc[globals()['cond_isocode{}'.format(i+1)]]

    globals()['cond_null{}'.format(i+1)] = (globals()['df_temp_isocode{}'.format(i+1)]['inv'].isnull() == True)

    globals()['df_temp_sa{}'.format(i+1)] = globals()['df_temp_isocode{}'.format(i+1)].loc[(globals()['cond_isocode{}'.format(i+1)] & ~globals()['cond_null{}'.format(i+1)])]

    df_temp.loc[(globals()['cond_isocode{}'.format(i+1)] & ~globals()['cond_null{}'.format(i+1)]), 'inv'] = stld(globals()['df_temp_sa{}'.format(i+1)]['inv'])

df_OECD_inv = df_temp

100%|██████████| 46/46 [00:00<00:00, 111.44it/s]


In [347]:
# merge df_OECD_inv with df_baseline

df_OECD_inv  = pd.merge(df_OECD_inv, df_baseline, how='right', on=['country', 'isocode', 'time']).drop(columns=['value'])

In [348]:
# df_OECD_inv

df_OECD_inv = df_OECD_inv.sort_values(by=['isocode', 'time']).reset_index(drop=True)
df_OECD_inv = df_OECD_inv.rename(columns={'inv':'OECD_inv'})
df_OECD_inv

,country,isocode,time,OECD_inv
0,Argentina,ARG,1950Q1,NaN
1,Argentina,ARG,1950Q2,NaN
2,Argentina,ARG,1950Q3,NaN
3,Argentina,ARG,1950Q4,NaN
4,Argentina,ARG,1951Q1,NaN
...,...,...,...,...
11771,South Africa,ZAF,2012Q4,NaN
11772,South Africa,ZAF,2013Q1,NaN
11773,South Africa,ZAF,2013Q2,NaN
11774,South Africa,ZAF,2013Q3,NaN


In [349]:
# set the data

data = df_OECD_inv

In [350]:
# OECD inv

period_var_46_countries('OECD_inv')

# list of replacement
# BEL : 1976:Q2-2013:Q4
# CAN : 1976:Q2-2013:Q4
# GBR : 1976:Q2-2013:Q4
# NLD : 1976:Q2-2013:Q4
# PRT : 1976:Q2-2013:Q4

,country,isocode,period,obs
0,Argentina,ARG,nan,nan
1,Australia,AUS,1976Q2-2013Q4,151
2,Austria,AUT,1984Q2-2013Q4,119
3,Belgium,BEL,1976Q2-2013Q4,151
4,Bulgaria,BGR,nan,nan
5,Brazil,BRA,nan,nan
6,Canada,CAN,1976Q2-2013Q4,151
7,Switzerland,CHE,1979Q2-2013Q4,139
8,Chile,CHL,2000Q2-2013Q4,55
9,Colombia,COL,1989Q2-2013Q4,99


# **11. data**

In [351]:
# generate the data

# ggdebt
data = pd.merge(data_221115, df_JME_WEO_ggdebt, how='outer', on=['country', 'isocode', 'time'])

# ggcon
data = pd.merge(data, df_OECD_ggcon, how='outer', on=['country', 'isocode', 'time'])

# gdp
data = pd.merge(data, df_OECD_gdp, how='outer', on=['country', 'isocode', 'time'])

# ca
data = pd.merge(data, df_OECD_ca, how='outer', on=['country', 'isocode', 'time'])
data = pd.merge(data, df_WEO_ca, how='outer', on=['country', 'isocode', 'time'])
data = pd.merge(data, df_COL_ca, how='outer', on=['country', 'isocode', 'time'])

# pcon
data = pd.merge(data, df_OECD_pcon, how='outer', on=['country', 'isocode', 'time'])
data = pd.merge(data, df_TUR, how='outer', on=['country', 'isocode', 'time'])

# inv
data = pd.merge(data, df_OECD_inv, how='outer', on=['country', 'isocode', 'time'])

data = data.sort_values(by=['isocode', 'time']).reset_index(drop=True)

In [352]:
# replace the value of variables

# OECD ggcon
OECD_ggcon_list = ['BEL', 'CAN', 'GBR', 'NLD', 'PRT']

for i in tqdm(range(0, len(OECD_ggcon_list))):

    globals()['cond_OECD_ggcon_{}'.format(i+1)] = (data['isocode'] == OECD_ggcon_list[i])
    data.loc[globals()['cond_OECD_ggcon_{}'.format(i+1)], 'ggcon'] = data['OECD_ggcon']

# TUR ggcon
TUR_ggcon_list = ['TUR']

for i in tqdm(range(0, len(TUR_ggcon_list))):

    globals()['cond_TUR_ggcon_{}'.format(i+1)] = (data['isocode'] == TUR_ggcon_list[i])
    data.loc[globals()['cond_TUR_ggcon_{}'.format(i+1)], 'ggcon'] = data['TUR_ggcon']

# OECD gdp
OECD_gdp_list = ['BEL', 'CAN', 'GBR', 'NLD', 'PRT']

for i in tqdm(range(0, len(OECD_gdp_list))):

    globals()['cond_OECD_gdp_{}'.format(i+1)] = (data['isocode'] == OECD_gdp_list[i])
    data.loc[globals()['cond_OECD_gdp_{}'.format(i+1)], 'gdp'] = data['OECD_gdp']

# TUR gdp
TUR_gdp_list = ['TUR']

for i in tqdm(range(0, len(TUR_gdp_list))):

    globals()['cond_TUR_gdp_{}'.format(i+1)] = (data['isocode'] == TUR_gdp_list[i])
    data.loc[globals()['cond_TUR_gdp_{}'.format(i+1)], 'gdp'] = data['TUR_gdp']

# OECD ca
OECD_ca_list = ['BEL', 'CAN', 'GBR', 'NLD', 'PRT']

for i in tqdm(range(0, len(OECD_ca_list))):

    globals()['cond_OECD_ca_{}'.format(i+1)] = (data['isocode'] == OECD_ca_list[i])
    data.loc[globals()['cond_OECD_ca_{}'.format(i+1)], 'ca'] = data['OECD_ca']

# COL ca
COL_ca_list = ['COL']

for i in tqdm(range(0, len(COL_ca_list))):

    globals()['cond_COL_ca_{}'.format(i+1)] = (data['isocode'] == COL_ca_list[i])
    data.loc[globals()['cond_COL_ca_{}'.format(i+1)], 'ca'] = data['COL_ca']

# OECD pcon
OECD_pcon_list = ['BEL', 'CAN', 'GBR', 'NLD', 'PRT']

for i in tqdm(range(0, len(OECD_pcon_list))):

    globals()['cond_OECD_pcon_{}'.format(i+1)] = (data['isocode'] == OECD_pcon_list[i])
    data.loc[globals()['cond_OECD_pcon_{}'.format(i+1)], 'pcon'] = data['OECD_pcon']

# TUR pcon
TUR_pcon_list = ['TUR']

for i in tqdm(range(0, len(TUR_pcon_list))):

    globals()['cond_TUR_pcon_{}'.format(i+1)] = (data['isocode'] == TUR_pcon_list[i])
    data.loc[globals()['cond_TUR_pcon_{}'.format(i+1)], 'pcon'] = data['TUR_pcon']

# OECD inv
OECD_inv_list = ['BEL', 'CAN', 'GBR', 'NLD', 'PRT']

for i in tqdm(range(0, len(OECD_inv_list))):

    globals()['cond_OECD_inv_{}'.format(i+1)] = (data['isocode'] == OECD_inv_list[i])
    data.loc[globals()['cond_OECD_inv_{}'.format(i+1)], 'inv'] = data['OECD_inv']

# TUR inv
TUR_inv_list = ['TUR']

for i in tqdm(range(0, len(TUR_inv_list))):

    globals()['cond_TUR_inv_{}'.format(i+1)] = (data['isocode'] == TUR_inv_list[i])
    data.loc[globals()['cond_TUR_inv_{}'.format(i+1)], 'inv'] = data['TUR_inv']

100%|██████████| 1/1 [00:00<00:00, 317.92it/s]


In [353]:
# data

data = data.sort_values(by=['isocode', 'time']).reset_index(drop=True)
data = data.rename(columns={'ggdebt':'ggdebt_q', 'JME_WEO_ggdebt':'ggdebt_a'})
data = data[['country', 'isocode', 'time', 'quarter',
             'ccode', 'region', 'income', 'oecd', 'g7',
             'ggdebt_a', 'ggdebt_q', 'ggcon', 'gdp', 'reer', 'ca', 'pcon', 'cds',
             'sample2', 'ex', 'inc_ex', 'debtlevel', 'inc_debt', 'tradeopen', 'inc_trade', 'fin', 'inc_fin', 'inc_fine', 'inc_debt_fine', 'finopen_debt', 'develop', 'inv', 'vehicle', 'vehicle2', 'adveme']]

data

,country,isocode,time,quarter,ccode,region,income,oecd,g7,ggdebt_a,...,fin,inc_fin,inc_fine,inc_debt_fine,finopen_debt,develop,inv,vehicle,vehicle2,adveme
0,Argentina,ARG,1950Q1,1,32,7,2,0,0,NaN,...,1,3,2,4,2,2,NaN,0,0,2
1,Argentina,ARG,1950Q2,2,32,7,2,0,0,NaN,...,1,3,2,4,2,2,NaN,0,0,2
2,Argentina,ARG,1950Q3,3,32,7,2,0,0,NaN,...,1,3,2,4,2,2,NaN,0,0,2
3,Argentina,ARG,1950Q4,4,32,7,2,0,0,NaN,...,1,3,2,4,2,2,NaN,0,0,2
4,Argentina,ARG,1951Q1,5,32,7,2,0,0,NaN,...,1,3,2,4,2,2,NaN,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11771,South Africa,ZAF,2012Q4,252,39,5,2,0,0,37.408,...,1,3,3,5,1,2,1.912744e+11,0,0,2
11772,South Africa,ZAF,2013Q1,253,39,5,2,0,0,40.356,...,1,3,3,5,1,2,1.943342e+11,0,0,2
11773,South Africa,ZAF,2013Q2,254,39,5,2,0,0,40.356,...,1,3,3,5,1,2,1.983998e+11,0,0,2
11774,South Africa,ZAF,2013Q3,255,39,5,2,0,0,40.356,...,1,3,3,5,1,2,2.026111e+11,0,0,2


In [364]:
# export the data

data.to_excel(excel_writer='data_230705.xlsx')

In [355]:
# set the data

data = data

In [356]:
# ggdebt_a

period_var_46_countries('ggdebt_a')

,country,isocode,period,obs
0,Argentina,ARG,1990Q1-2013Q4,96
1,Australia,AUS,1980Q1-2013Q4,136
2,Austria,AUT,1988Q1-2013Q4,104
3,Belgium,BEL,1980Q1-2013Q4,136
4,Bulgaria,BGR,1991Q1-2013Q4,92
5,Brazil,BRA,1991Q1-2013Q4,92
6,Canada,CAN,1980Q1-2013Q4,136
7,Switzerland,CHE,1990Q1-2013Q4,96
8,Chile,CHL,1991Q1-2013Q4,92
9,Colombia,COL,1990Q1-2013Q4,96


In [357]:
# ggdebt_q

period_var_46_countries('ggdebt_q')

,country,isocode,period,obs
0,Argentina,ARG,1992Q4-2013Q4,85
1,Australia,AUS,1988Q2-2013Q4,103
2,Austria,AUT,2000Q1-2013Q4,56
3,Belgium,BEL,1995Q1-2013Q4,76
4,Bulgaria,BGR,2000Q1-2013Q4,56
5,Brazil,BRA,1998Q1-2013Q4,64
6,Canada,CAN,1990Q1-2013Q4,96
7,Switzerland,CHE,1995Q1-2013Q4,76
8,Chile,CHL,1998Q1-2013Q4,64
9,Colombia,COL,1996Q4-2013Q4,69


In [358]:
# gdp

period_var_46_countries('gdp')

,country,isocode,period,obs
0,Argentina,ARG,2004Q1-2013Q4,40
1,Australia,AUS,1980Q1-2013Q4,136
2,Austria,AUT,1995Q1-2013Q4,76
3,Belgium,BEL,1976Q2-2013Q4,151
4,Bulgaria,BGR,1999Q1-2013Q4,60
5,Brazil,BRA,1996Q1-2013Q4,72
6,Canada,CAN,1976Q2-2013Q4,151
7,Switzerland,CHE,1980Q1-2013Q4,136
8,Chile,CHL,1996Q1-2013Q4,72
9,Colombia,COL,2005Q1-2013Q4,36


In [359]:
# ggcon

period_var_46_countries('ggcon')

,country,isocode,period,obs
0,Argentina,ARG,2004Q1-2013Q4,40
1,Australia,AUS,1980Q1-2013Q4,136
2,Austria,AUT,1995Q1-2013Q4,76
3,Belgium,BEL,1976Q2-2013Q4,151
4,Bulgaria,BGR,1999Q1-2013Q4,60
5,Brazil,BRA,1996Q1-2013Q4,72
6,Canada,CAN,1976Q2-2013Q4,151
7,Switzerland,CHE,1980Q1-2013Q4,136
8,Chile,CHL,1996Q1-2013Q4,72
9,Colombia,COL,2005Q1-2013Q4,36


In [360]:
# pcon

period_var_46_countries('pcon')

,country,isocode,period,obs
0,Argentina,ARG,2004Q1-2013Q4,40
1,Australia,AUS,1980Q1-2013Q4,136
2,Austria,AUT,1995Q1-2013Q4,76
3,Belgium,BEL,1976Q2-2013Q4,151
4,Bulgaria,BGR,1999Q1-2013Q4,60
5,Brazil,BRA,1996Q1-2013Q4,72
6,Canada,CAN,1976Q2-2013Q4,151
7,Switzerland,CHE,1980Q1-2013Q4,136
8,Chile,CHL,1996Q1-2013Q4,72
9,Colombia,COL,2005Q1-2013Q4,36


In [361]:
# inv

period_var_46_countries('inv')

,country,isocode,period,obs
0,Argentina,ARG,2004Q1-2013Q4,40
1,Australia,AUS,1959Q3-2013Q4,218
2,Austria,AUT,1995Q1-2013Q4,76
3,Belgium,BEL,1976Q2-2013Q4,151
4,Bulgaria,BGR,1995Q1-2013Q4,76
5,Brazil,BRA,1996Q1-2013Q4,72
6,Canada,CAN,1976Q2-2013Q4,151
7,Switzerland,CHE,1980Q1-2013Q4,136
8,Chile,CHL,1996Q1-2013Q4,72
9,Colombia,COL,2005Q1-2013Q4,36


In [362]:
# reer

period_var_46_countries('reer')

,country,isocode,period,obs
0,Argentina,ARG,1994Q1-2013Q4,80
1,Australia,AUS,1980Q1-2013Q4,136
2,Austria,AUT,1979Q1-2013Q4,140
3,Belgium,BEL,1991Q1-2013Q4,92
4,Bulgaria,BGR,1999Q1-2013Q4,60
5,Brazil,BRA,1980Q1-2013Q4,136
6,Canada,CAN,1980Q1-2013Q4,136
7,Switzerland,CHE,1979Q1-2013Q4,140
8,Chile,CHL,1989Q1-2013Q4,100
9,Colombia,COL,1980Q1-2013Q4,136


In [363]:
# ca

period_var_46_countries('ca')

,country,isocode,period,obs
0,Argentina,ARG,2004Q1-2013Q4,40
1,Australia,AUS,1980Q1-2013Q4,136
2,Austria,AUT,1984Q1-2013Q4,120
3,Belgium,BEL,1989Q2-2013Q4,99
4,Bulgaria,BGR,1999Q1-2013Q4,60
5,Brazil,BRA,1996Q1-2013Q4,72
6,Canada,CAN,1976Q2-2013Q4,151
7,Switzerland,CHE,1985Q1-2013Q4,116
8,Chile,CHL,1996Q1-2013Q4,72
9,Colombia,COL,1996Q1-2013Q4,72
